In [ ]:
# from langchain_community.document_loaders import PyMuPDFLoader

# # Use only PyPDFLoader - most reliable
# loader = PyPDFLoader(r"D:\Upwork\POC\RAG\Data\Financial results_bajaj.pdf")
# pdf_docs = loader.load()
# print(f"Loaded {len(pdf_docs)} pages")

Loaded 18 pages


In [26]:
# import pandas as pd
# from langchain_community.document_loaders import PyMuPDFLoader
# import fitz  # PyMuPDF

# # Load PDF with rotation handling
# loader = PyMuPDFLoader(r"D:\Upwork\POC\RAG\Data\Financial results_bajaj.pdf")
# docs = loader.load()

# # Extract content with proper orientation
# for i, doc in enumerate(docs):
#     print(f"\n--- Page {i+1} ---")
    
#     # Get raw PDF page to check rotation
#     pdf_document = fitz.open(r"D:\Upwork\POC\RAG\Data\Financial results_bajaj.pdf")
#     page = pdf_document[i]
    
#     # Detect and correct rotation
#     if page.rotation != 0:
#         print(f"Page rotated by {page.rotation} degrees")
    
#     # Extract tables with rotation handling
#     tabs = page.find_tables()
#     if tabs.tables:
#         print(f"Found {len(tabs.tables)} table(s)")
#         for j, tab in enumerate(tabs.tables):
#             df = tab.to_pandas()
#             print(f"Table {j+1}:")
#             print(df.head(8))
#     else:
#         print("Text content:")
#         print(doc.page_content[:500])
    
#     pdf_document.close()

In [29]:
import pandas as pd
import fitz
import re
from langchain_community.document_loaders import PyMuPDFLoader

# Load PDF
loader = PyMuPDFLoader(r"D:\Upwork\POC\RAG\Data\Financial results_bajaj.pdf")
docs = loader.load()

pdf_document = fitz.open(r"D:\Upwork\POC\RAG\Data\Financial results_bajaj.pdf")

for i, page in enumerate(pdf_document):
    print(f"\n--- Page {i+1} ---")
    
    # Check orientation
    if page.rotation != 0:
        print(f"Orientation: Rotated {page.rotation}°")
    
    # Extract tables first
    tabs = page.find_tables()
    if tabs.tables:
        print(f"Found {len(tabs.tables)} table(s)")
        
        for j, tab in enumerate(tabs.tables):
            df = tab.to_pandas()
            
            # Clean noise: remove dots, special chars, and fix spacing
            df = df.map(lambda x: re.sub(r'[•·\.\s]+$', '', str(x)) if pd.notna(x) else x)
            df = df.map(lambda x: re.sub(r'[^\w\s\.\,\-\+\%\$\d]', '', str(x)) if pd.notna(x) else x)
            df = df.map(lambda x: re.sub(r'\s+', ' ', str(x)).strip() if pd.notna(x) else x)
            
            # Remove empty rows and columns
            df = df.dropna(how='all').dropna(axis=1, how='all')
            
            # Handle multi-header by checking first few rows
            if len(df) > 2:
                # Check if first two rows form a multi-header
                non_empty_count = sum(df.iloc[0].notna() & df.iloc[1].notna())
                if non_empty_count > len(df.columns) * 0.5:  # If >50% columns have data in both rows
                    # Merge first two rows as header
                    new_header = []
                    for col1, col2 in zip(df.iloc[0], df.iloc[1]):
                        merged = f"{col1} {col2}".strip()
                        new_header.append(merged if merged else col1 or col2)
                    df.columns = new_header
                    df = df[2:].reset_index(drop=True)
                    print("Multi-header processed")
            
            print(f"Table {j+1}:")
            print(df)
            print("\n" + "="*50)
    else:
        # Clean text content
        text = docs[i].page_content
        clean_text = re.sub(r'[•·\*]+', '', text)  # Remove bullet dots
        clean_text = re.sub(r'\n\s*\n', '\n', clean_text)  # Remove empty lines
        clean_text = re.sub(r'\s+', ' ', clean_text).strip()  # Fix spacing
        
        print("Cleaned Text Content:")
        print(clean_text[:800])

pdf_document.close()


--- Page 1 ---
Found 1 table(s)
Table 1:
                                                Col0  \
0  To, The Manager Listing Department SSE Limited...   
1                                    SSE Code 544252   

                                        23 July 2025  
0  To, The Manager Listing Department National St...  
1                             NSE Code BAJAJHFL - EQ  


--- Page 2 ---
Cleaned Text Content:
Singhi & Co. Chartered Accountants B2 402B, Marathon lnnova, 4th Floor Off Ganpatrao Kadam Marg, Lower Parel, Mumbai-400013, India Mukund M. Chitale & Co. Chartered Accountants 2nd Floor, Kapur House, Paranjape B Scheme Road No. 1, Vile Parle (E), Mumbai - 400 057, India Independent Auditors' Limited Review Report on the Quarterly Unaudited Financial Results of Bajaj Housing Finance Limited pursuant to the Regulation 33 and 52 of the SEBI {Listing Obligations and Disclosure Requirements) Regulations, 2015, {as amended) To the Board of Directors of Bajaj Housing Finance Limited 1

In [49]:
# import fitz  # PyMuPDF
# import cv2
# import numpy as np
# from PIL import Image, ImageDraw
# import os
# import io

# def process_pdf_blocks(pdf_path, output_path, specific_page=None):
#     """Extract blocks from PDF and create transparent overlay"""
#     doc = fitz.open(pdf_path)
    
#     # Validate page number if specific page is requested
#     if specific_page is not None:
#         if specific_page < 0 or specific_page >= len(doc):
#             print(f"Error: Page {specific_page} is out of range. PDF has {len(doc)} pages.")
#             doc.close()
#             return
    
#     print(f"Processing PDF: {os.path.basename(pdf_path)}")
#     print(f"Total pages: {len(doc)}")
    
#     if specific_page is not None:
#         # Process only the specific page
#         pages_to_process = [specific_page]
#         print(f"Processing only page {specific_page}")
#     else:
#         # Process all pages
#         pages_to_process = range(len(doc))
#         print("Processing all pages")
    
#     for page_num in pages_to_process:
#         page = doc[page_num]
        
#         # Get text blocks with their coordinates
#         blocks = page.get_text("dict")
        
#         # Create a transparent overlay
#         overlay = create_transparent_overlay(page, blocks, page_num)
        
#         # Save the overlay
#         if specific_page is not None:
#             output_filename = f"{output_path}_page_{page_num}_overlay.png"
#         else:
#             output_filename = f"{output_path}_page_{page_num}_overlay.png"
        
#         overlay.save(output_filename)
#         print(f"Saved overlay for page {page_num}: {output_filename}")
        
#         # Print block information
#         print(f"\nPage {page_num} - Found {len(blocks['blocks'])} blocks:")
#         for i, block in enumerate(blocks['blocks']):
#             if 'bbox' in block:
#                 print(f"  Block {i+1}: {block['bbox']} - Type: {block.get('type', 'unknown')}")
    
#     doc.close()

# def create_transparent_overlay(page, blocks, page_num):
#     """Create transparent overlay for PDF blocks"""
#     # Get page dimensions
#     mat = fitz.Matrix(2, 2)  # Zoom factor 2 for better quality
#     pix = page.get_pixmap(matrix=mat)
    
#     # Convert to PIL Image
#     img_data = pix.tobytes("png")
#     img = Image.open(io.BytesIO(img_data))
    
#     # Create transparent overlay
#     overlay = Image.new("RGBA", img.size, (0, 0, 0, 0))
#     draw = ImageDraw.Draw(overlay)
    
#     # Define colors for different block types
#     colors = {
#         'text': (255, 0, 0, 64),      # Red for text
#         'image': (0, 255, 0, 64),     # Green for images
#         'unknown': (0, 0, 255, 64)    # Blue for unknown
#     }
    
#     block_count = 0
#     for block in blocks['blocks']:
#         if 'bbox' in block:
#             # Scale bbox coordinates to match the high-res image
#             bbox = [coord * 2 for coord in block['bbox']]
            
#             # Determine block type and color
#             block_type = block.get('type', 'unknown')
#             color = colors.get(block_type, colors['unknown'])
            
#             # Draw semi-transparent rectangle
#             draw.rectangle(bbox, fill=color, outline=(color[0], color[1], color[2], 255), width=2)
            
#             # Add block number
#             draw.text((bbox[0] + 5, bbox[1] + 5), str(block_count + 1), 
#                      fill=(0, 0, 0, 255))
            
#             block_count += 1
    
#     # Combine original image with overlay
#     result = Image.alpha_composite(img.convert("RGBA"), overlay)
#     return result

# def get_user_choice():
#     """Get user input for page selection"""
#     print("\n" + "="*50)
#     print("PDF Block Detection Tool")
#     print("="*50)
    
#     while True:
#         choice = input("\nDo you want to process a specific page or all pages?\n"
#                       "1. Specific page\n"
#                       "2. All pages\n"
#                       "3. Exit\n"
#                       "Enter your choice (1/2/3): ").strip()
        
#         if choice == '1':
#             try:
#                 page_num = int(input("Enter the page number (0-based index): "))
#                 return page_num
#             except ValueError:
#                 print("Please enter a valid number!")
#                 continue
#         elif choice == '2':
#             return None
#         elif choice == '3':
#             return 'exit'
#         else:
#             print("Invalid choice! Please enter 1, 2, or 3.")

# def display_pdf_info(pdf_path):
#     """Display basic information about the PDF"""
#     try:
#         doc = fitz.open(pdf_path)
#         print(f"\nPDF Information:")
#         print(f"  File: {os.path.basename(pdf_path)}")
#         print(f"  Total pages: {len(doc)}")
#         print(f"  File size: {os.path.getsize(pdf_path) / 1024:.2f} KB")
#         doc.close()
#     except Exception as e:
#         print(f"Error reading PDF: {e}")

# def main():
#     """Main function to run the PDF processing"""
#     pdf_path = r"D:\Upwork\POC\RAG\Data\Financial results_bajaj.pdf"
    
#     # Check if file exists
#     if not os.path.exists(pdf_path):
#         print(f"Error: File not found - {pdf_path}")
#         return
    
#     # Display PDF information
#     display_pdf_info(pdf_path)
    
#     # Create output directory
#     output_dir = r"D:\Upwork\POC\RAG\Data\output"
#     os.makedirs(output_dir, exist_ok=True)
    
#     # Get user choice
#     while True:
#         user_choice = get_user_choice()
        
#         if user_choice == 'exit':
#             print("Goodbye!")
#             break
        
#         # Process the PDF based on user choice
#         output_base = os.path.join(output_dir, "financial_results_bajaj")
#         process_pdf_blocks(pdf_path, output_base, user_choice)
        
#         # Ask if user wants to process another page
#         another = input("\nDo you want to process another page? (y/n): ").strip().lower()
#         if another != 'y':
#             print("Processing completed!")
#             break

# # Alternative function for batch processing
# def batch_process_pdf(pdf_path, output_dir, pages=None):
#     """Batch process PDF with optional specific pages"""
#     if not os.path.exists(pdf_path):
#         print(f"Error: File not found - {pdf_path}")
#         return
    
#     os.makedirs(output_dir, exist_ok=True)
    
#     doc = fitz.open(pdf_path)
#     total_pages = len(doc)
    
#     if pages is None:
#         pages_to_process = range(total_pages)
#         print(f"Processing all {total_pages} pages")
#     else:
#         # Validate page numbers
#         valid_pages = [p for p in pages if 0 <= p < total_pages]
#         invalid_pages = [p for p in pages if p not in valid_pages]
        
#         if invalid_pages:
#             print(f"Warning: Invalid page numbers ignored: {invalid_pages}")
        
#         pages_to_process = valid_pages
#         print(f"Processing specific pages: {pages_to_process}")
    
#     for page_num in pages_to_process:
#         output_base = os.path.join(output_dir, f"page_{page_num}")
#         process_pdf_blocks(pdf_path, output_base, page_num)
    
#     doc.close()

# # Example usage functions
# def example_usage():
#     """Show examples of how to use the functions"""
#     pdf_path = r"D:\Upwork\POC\RAG\Data\Financial results_bajaj.pdf"
#     output_dir = r"D:\Upwork\POC\RAG\Data\output"
    
#     print("Example Usage Patterns:")
#     print("1. Process all pages:")
#     print("   batch_process_pdf(pdf_path, output_dir)")
    
#     print("\n2. Process specific pages (e.g., page 0 and 2):")
#     print("   batch_process_pdf(pdf_path, output_dir, pages=[0, 2])")
    
#     print("\n3. Process single page (page 1):")
#     print("   batch_process_pdf(pdf_path, output_dir, pages=[1])")
    
#     print("\n4. Interactive mode:")
#     print("   main()")

# if __name__ == "__main__":
#     # Run the interactive main function
#     main()
    
#     # Uncomment the line below for batch processing examples
#     # example_usage()

In [46]:
# import fitz  # PyMuPDF
# import cv2
# import numpy as np
# from PIL import Image, ImageDraw
# import os
# import io
# from collections import Counter
# import re

# def process_pdf_blocks(pdf_path, output_path, specific_page=None):
#     """Extract different types of blocks from PDF and create color-coded transparent overlay"""
#     doc = fitz.open(pdf_path)
    
#     # Validate page number if specific page is requested
#     if specific_page is not None:
#         if specific_page < 0 or specific_page >= len(doc):
#             print(f"Error: Page {specific_page} is out of range. PDF has {len(doc)} pages.")
#             doc.close()
#             return
    
#     print(f"Processing PDF: {os.path.basename(pdf_path)}")
#     print(f"Total pages: {len(doc)}")
    
#     if specific_page is not None:
#         pages_to_process = [specific_page]
#         print(f"Processing only page {specific_page}")
#     else:
#         pages_to_process = range(len(doc))
#         print("Processing all pages")
    
#     for page_num in pages_to_process:
#         page = doc[page_num]
        
#         # Get all blocks from the page
#         raw_blocks = page.get_text("dict")
        
#         # Classify blocks into different types
#         classified_blocks = classify_blocks(raw_blocks, page)
        
#         # Create a color-coded transparent overlay
#         overlay = create_color_coded_overlay(page, classified_blocks, page_num)
        
#         # Save the overlay
#         if specific_page is not None:
#             output_filename = f"{output_path}_page_{page_num}_color_coded.png"
#         else:
#             output_filename = f"{output_path}_page_{page_num}_color_coded.png"
        
#         overlay.save(output_filename)
#         print(f"Saved color-coded overlay for page {page_num}: {output_filename}")
        
#         # Print block statistics
#         print_block_statistics(classified_blocks, page_num)
    
#     doc.close()

# def classify_blocks(raw_blocks, page):
#     """Classify blocks into different types: text, logos, stamps, tables, headers"""
#     classified_blocks = {
#         'text_blocks': [],
#         'logos': [],
#         'stamps': [],
#         'tables_no_header': [],
#         'tables_single_header': [],
#         'tables_multi_header': [],
#         'headers_single': [],
#         'headers_multi': [],
#         'unknown': []
#     }
    
#     for block in raw_blocks['blocks']:
#         block_type = identify_block_type(block, page)
#         classified_blocks[block_type].append(block)
    
#     return classified_blocks

# def identify_block_type(block, page):
#     """Identify the type of block based on its characteristics"""
    
#     # Check if it's an image block
#     if block.get('type') == 1:  # Image block
#         return identify_image_type(block, page)
    
#     # Text block analysis
#     if 'lines' in block and block['lines']:
#         text_content = extract_text_from_block(block).strip()
        
#         # Skip empty blocks
#         if not text_content:
#             return 'unknown'
        
#         # Check for headers
#         header_type = identify_header_type(block, text_content)
#         if header_type:
#             return header_type
        
#         # Check for tables
#         table_type = identify_table_type(block, text_content)
#         if table_type:
#             return table_type
        
#         # Check for stamps
#         if is_stamp(block, text_content):
#             return 'stamps'
        
#         # Default to text block
#         return 'text_blocks'
    
#     return 'unknown'

# def identify_image_type(block, page):
#     """Identify if image is logo or other type"""
#     bbox = block.get('bbox', [])
#     if bbox:
#         width = bbox[2] - bbox[0]
#         height = bbox[3] - bbox[1]
        
#         # Logo characteristics: often smaller, square-ish, positioned at top corners
#         if width < 200 and height < 200:
#             # Check position (logos are often in corners)
#             page_width = page.rect.width
#             page_height = page.rect.height
            
#             if (bbox[0] < page_width * 0.2 or bbox[2] > page_width * 0.8) and \
#                (bbox[1] < page_height * 0.2):
#                 return 'logos'
        
#         # Stamp characteristics: often circular or rectangular with specific content
#         if 50 < width < 300 and 50 < height < 300:
#             return 'stamps'
    
#     return 'unknown'

# def identify_header_type(block, text_content):
#     """Identify header blocks"""
#     lines = block.get('lines', [])
    
#     if not lines:
#         return None
    
#     # Analyze font characteristics
#     font_sizes = []
#     is_bold = False
    
#     for line in lines:
#         for span in line.get('spans', []):
#             font_size = span.get('size', 0)
#             font_sizes.append(font_size)
#             font_flags = span.get('flags', 0)
#             if font_flags & 2:  # Bold flag
#                 is_bold = True
    
#     avg_font_size = sum(font_sizes) / len(font_sizes) if font_sizes else 0
    
#     # Header detection logic
#     if len(lines) == 1:
#         # Single line header
#         if (avg_font_size > 12 and is_bold) or \
#            text_content.isupper() or \
#            len(text_content.split()) <= 5:
#             return 'headers_single'
#     else:
#         # Multi-line header
#         if avg_font_size > 10 and (is_bold or text_content.isupper()):
#             return 'headers_multi'
    
#     return None

# def identify_table_type(block, text_content):
#     """Identify table blocks"""
#     lines = block.get('lines', [])
    
#     if len(lines) < 2:  # Tables typically have multiple lines
#         return None
    
#     # Check for tabular structure
#     has_tabular_pattern = check_tabular_pattern(block)
#     line_heights = analyze_line_structure(block)
    
#     if has_tabular_pattern:
#         # Analyze header presence
#         header_analysis = analyze_table_headers(block)
        
#         if header_analysis == 'no_header':
#             return 'tables_no_header'
#         elif header_analysis == 'single_header':
#             return 'tables_single_header'
#         elif header_analysis == 'multi_header':
#             return 'tables_multi_header'
    
#     return None

# def check_tabular_pattern(block):
#     """Check if block has tabular pattern"""
#     lines = block.get('lines', [])
    
#     if len(lines) < 3:
#         return False
    
#     # Check for consistent spacing (indicating columns)
#     x_positions = []
#     for line in lines:
#         for span in line.get('spans', []):
#             x_positions.append(span['bbox'][0])
    
#     # Count unique x positions (potential columns)
#     unique_x = len(set(round(x) for x in x_positions))
    
#     return unique_x >= 3  # At least 3 potential columns

# def analyze_line_structure(block):
#     """Analyze line structure for table detection"""
#     lines = block.get('lines', [])
#     heights = []
    
#     for line in lines:
#         bbox = line.get('bbox', [0, 0, 0, 0])
#         height = bbox[3] - bbox[1]
#         heights.append(height)
    
#     return heights

# def analyze_table_headers(block):
#     """Analyze table headers"""
#     lines = block.get('lines', [])
    
#     if not lines:
#         return 'no_header'
    
#     # Check first line for header characteristics
#     first_line = lines[0]
#     first_line_text = extract_text_from_line(first_line).strip()
    
#     header_spans = first_line.get('spans', [])
#     if header_spans:
#         avg_font_size = sum(span.get('size', 0) for span in header_spans) / len(header_spans)
#         is_bold = any(span.get('flags', 0) & 2 for span in header_spans)
        
#         # Check if first line looks like a header
#         if (avg_font_size > 10 and is_bold) or first_line_text.isupper():
#             # Check if second line also looks like a header
#             if len(lines) > 1:
#                 second_line = lines[1]
#                 second_line_text = extract_text_from_line(second_line).strip()
#                 second_spans = second_line.get('spans', [])
                
#                 if second_spans:
#                     second_avg_font = sum(span.get('size', 0) for span in second_spans) / len(second_spans)
#                     second_bold = any(span.get('flags', 0) & 2 for span in second_spans)
                    
#                     if (second_avg_font > 10 and second_bold) or second_line_text.isupper():
#                         return 'multi_header'
            
#             return 'single_header'
    
#     return 'no_header'

# def is_stamp(block, text_content):
#     """Identify stamp blocks"""
#     # Stamp patterns: dates, signatures, approvals, etc.
#     stamp_patterns = [
#         r'\b\d{1,2}[-/]\d{1,2}[-/]\d{2,4}\b',  # Dates
#         r'\b(?:approved|signed|verified|confirmed)\b',
#         r'\b(?:signature|stamp|seal)\b',
#         r'\b[A-Z]{2,}\b',  # All caps words (common in stamps)
#     ]
    
#     for pattern in stamp_patterns:
#         if re.search(pattern, text_content, re.IGNORECASE):
#             return True
    
#     return False

# def extract_text_from_block(block):
#     """Extract text content from a block"""
#     text_parts = []
#     for line in block.get('lines', []):
#         text_parts.append(extract_text_from_line(line))
#     return ' '.join(text_parts)

# def extract_text_from_line(line):
#     """Extract text content from a line"""
#     text_parts = []
#     for span in line.get('spans', []):
#         text_parts.append(span.get('text', ''))
#     return ' '.join(text_parts)

# def create_color_coded_overlay(page, classified_blocks, page_num):
#     """Create color-coded transparent overlay for different block types"""
#     # Get page dimensions
#     mat = fitz.Matrix(2, 2)  # Zoom factor 2 for better quality
#     pix = page.get_pixmap(matrix=mat)
    
#     # Convert to PIL Image
#     img_data = pix.tobytes("png")
#     img = Image.open(io.BytesIO(img_data))
    
#     # Create transparent overlay
#     overlay = Image.new("RGBA", img.size, (0, 0, 0, 0))
#     draw = ImageDraw.Draw(overlay)
    
#     # Define colors for different block types (RGBA)
#     colors = {
#         'text_blocks': (100, 150, 255, 64),        # Light Blue - Regular text
#         'logos': (255, 100, 100, 128),             # Light Red - Logos
#         'stamps': (255, 200, 50, 128),             # Yellow - Stamps
#         'tables_no_header': (100, 255, 100, 64),   # Light Green - Tables without headers
#         'tables_single_header': (50, 200, 50, 96), # Green - Tables with single header
#         'tables_multi_header': (0, 150, 0, 128),   # Dark Green - Tables with multiple headers
#         'headers_single': (255, 100, 255, 96),     # Pink - Single line headers
#         'headers_multi': (200, 50, 200, 128),      # Purple - Multi-line headers
#         'unknown': (128, 128, 128, 64)             # Gray - Unknown
#     }
    
#     # Labels for block types
#     labels = {
#         'text_blocks': 'TEXT',
#         'logos': 'LOGO',
#         'stamps': 'STAMP',
#         'tables_no_header': 'TABLE_NO_HDR',
#         'tables_single_header': 'TABLE_1_HDR',
#         'tables_multi_header': 'TABLE_MULTI_HDR',
#         'headers_single': 'HEADER_1',
#         'headers_multi': 'HEADER_MULTI',
#         'unknown': 'UNKNOWN'
#     }
    
#     # Draw blocks for each type
#     block_counter = 0
#     for block_type, blocks in classified_blocks.items():
#         color = colors.get(block_type, colors['unknown'])
#         label = labels.get(block_type, 'UNKNOWN')
        
#         for block in blocks:
#             if 'bbox' in block:
#                 # Scale bbox coordinates to match the high-res image
#                 bbox = [coord * 2 for coord in block['bbox']]
                
#                 # Draw semi-transparent rectangle
#                 draw.rectangle(bbox, fill=color, 
#                               outline=(color[0], color[1], color[2], 255), 
#                               width=3)
                
#                 # Add block type label
#                 draw.text((bbox[0] + 5, bbox[1] + 5), f"{label}_{block_counter}", 
#                          fill=(0, 0, 0, 255))
                
#                 block_counter += 1
    
#     # Add legend
#     add_legend(draw, img.size, colors, labels)
    
#     # Combine original image with overlay
#     result = Image.alpha_composite(img.convert("RGBA"), overlay)
#     return result

# def add_legend(draw, image_size, colors, labels):
#     """Add legend to the overlay"""
#     legend_x = 10
#     legend_y = 10
#     line_height = 25
    
#     # Draw legend background
#     draw.rectangle([legend_x, legend_y, legend_x + 250, 
#                    legend_y + len(colors) * line_height + 10], 
#                   fill=(255, 255, 255, 200), outline=(0, 0, 0, 255))
    
#     # Add legend title
#     draw.text((legend_x + 5, legend_y + 5), "Block Types Legend", 
#              fill=(0, 0, 0, 255))
    
#     # Add each legend item
#     for i, (block_type, color) in enumerate(colors.items()):
#         y_pos = legend_y + 30 + i * line_height
        
#         # Color box
#         draw.rectangle([legend_x + 10, y_pos, legend_x + 30, y_pos + 15], 
#                       fill=color, outline=(0, 0, 0, 255))
        
#         # Label text
#         label = labels.get(block_type, block_type)
#         draw.text((legend_x + 35, y_pos), label, fill=(0, 0, 0, 255))

# def print_block_statistics(classified_blocks, page_num):
#     """Print statistics about detected blocks"""
#     print(f"\nPage {page_num} - Block Statistics:")
#     print("-" * 40)
    
#     total_blocks = sum(len(blocks) for blocks in classified_blocks.values())
    
#     for block_type, blocks in classified_blocks.items():
#         count = len(blocks)
#         if count > 0:
#             percentage = (count / total_blocks) * 100
#             print(f"  {block_type:20}: {count:3} blocks ({percentage:5.1f}%)")
    
#     print(f"  {'Total':20}: {total_blocks:3} blocks")

# def get_user_choice():
#     """Get user input for page selection"""
#     print("\n" + "="*60)
#     print("PDF Block Detection & Classification Tool")
#     print("="*60)
    
#     while True:
#         choice = input("\nDo you want to process a specific page or all pages?\n"
#                       "1. Specific page\n"
#                       "2. All pages\n"
#                       "3. Exit\n"
#                       "Enter your choice (1/2/3): ").strip()
        
#         if choice == '1':
#             try:
#                 page_num = int(input("Enter the page number (0-based index): "))
#                 return page_num
#             except ValueError:
#                 print("Please enter a valid number!")
#                 continue
#         elif choice == '2':
#             return None
#         elif choice == '3':
#             return 'exit'
#         else:
#             print("Invalid choice! Please enter 1, 2, or 3.")

# def main():
#     """Main function to run the PDF processing"""
#     pdf_path = r"D:\Upwork\POC\RAG\Data\Financial results_bajaj.pdf"
    
#     # Check if file exists
#     if not os.path.exists(pdf_path):
#         print(f"Error: File not found - {pdf_path}")
#         return
    
#     # Create output directory
#     output_dir = r"D:\Upwork\POC\RAG\Data\output"
#     os.makedirs(output_dir, exist_ok=True)
    
#     # Get user choice
#     while True:
#         user_choice = get_user_choice()
        
#         if user_choice == 'exit':
#             print("Goodbye!")
#             break
        
#         # Process the PDF based on user choice
#         output_base = os.path.join(output_dir, "financial_results_bajaj")
#         process_pdf_blocks(pdf_path, output_base, user_choice)
        
#         # Ask if user wants to process another page
#         another = input("\nDo you want to process another page? (y/n): ").strip().lower()
#         if another != 'y':
#             print("Processing completed!")
#             break

# if __name__ == "__main__":
#     main()

In [52]:
# import fitz  # PyMuPDF
# import cv2
# import numpy as np
# from PIL import Image, ImageDraw, ImageFont
# import os
# import io
# import json
# from datetime import datetime

# def process_pdf_blocks_with_tables(pdf_path, output_path, specific_page=None, zoom_factor=2, save_json=False):
#     """Extract blocks from PDF and create transparent overlay with table detection"""
#     doc = fitz.open(pdf_path)
    
#     # Validate page number if specific page is requested
#     if specific_page is not None:
#         if specific_page < 0 or specific_page >= len(doc):
#             print(f"Error: Page {specific_page} is out of range. PDF has {len(doc)} pages.")
#             doc.close()
#             return
    
#     print(f"Processing PDF: {os.path.basename(pdf_path)}")
#     print(f"Total pages: {len(doc)}")
    
#     if specific_page is not None:
#         pages_to_process = [specific_page]
#         print(f"Processing only page {specific_page}")
#     else:
#         pages_to_process = range(len(doc))
#         print("Processing all pages")
    
#     all_blocks_data = {}
#     table_pages = []  # Track which pages have tables
    
#     for page_num in pages_to_process:
#         page = doc[page_num]
        
#         # Get text blocks with their coordinates
#         blocks = page.get_text("dict")
        
#         # Enhanced block analysis with table detection
#         analyzed_blocks, table_blocks = analyze_blocks_with_tables(blocks, page)
#         all_blocks_data[f"page_{page_num}"] = analyzed_blocks
        
#         # Check if page has tables
#         if table_blocks:
#             table_pages.append(page_num)
#             print(f"✅ Page {page_num}: Found {len(table_blocks)} table(s)")
        
#         # Create overlay with table highlighting
#         overlay = create_overlay_with_tables(page, analyzed_blocks, table_blocks, page_num, zoom_factor)
        
#         # Save the overlay
#         if specific_page is not None:
#             output_filename = f"{output_path}_page_{page_num}_overlay.png"
#         else:
#             output_filename = f"{output_path}_page_{page_num}_overlay.png"
        
#         overlay.save(output_filename)
#         print(f"Saved overlay for page {page_num}: {output_filename}")
        
#         # Print block information
#         print_block_statistics_with_tables(analyzed_blocks, table_blocks, page_num)
    
#     # Print table summary
#     if table_pages:
#         print(f"\n🎯 TABLE SUMMARY: Tables found on {len(table_pages)} page(s): {table_pages}")
#     else:
#         print(f"\n❌ No tables detected in the document")
    
#     # Save JSON data if requested
#     if save_json:
#         json_output = f"{os.path.splitext(output_path)[0]}_blocks_data.json"
#         with open(json_output, 'w', encoding='utf-8') as f:
#             json.dump(all_blocks_data, f, indent=2, ensure_ascii=False)
#         print(f"Saved block data to: {json_output}")
    
#     doc.close()
#     return all_blocks_data, table_pages

# def analyze_blocks_with_tables(blocks, page):
#     """Enhanced block analysis with table detection"""
#     analyzed_blocks = []
#     table_blocks = []
    
#     # Method 1: Detect tables using PyMuPDF's built-in table finder
#     tables_pymupdf = detect_tables_pymupdf(page)
    
#     # Method 2: Detect tables from text structure
#     tables_text_based = detect_tables_from_text(blocks)
    
#     # Combine tables from both methods
#     all_tables = tables_pymupdf + tables_text_based
    
#     # Remove duplicate tables (overlapping regions)
#     unique_tables = remove_duplicate_tables(all_tables)
    
#     for i, block in enumerate(blocks['blocks']):
#         block_info = {
#             'block_number': i + 1,
#             'bbox': block.get('bbox', []),
#             'type': block.get('type', 'unknown'),
#             'subtype': 'unknown',
#             'is_table': False
#         }
        
#         # Check if this block is part of a table
#         for table in unique_tables:
#             if is_block_in_table(block_info['bbox'], table['bbox']):
#                 block_info['is_table'] = True
#                 block_info['table_id'] = table['id']
#                 if table not in table_blocks:
#                     table_blocks.append(table)
#                 break
        
#         # Enhanced type detection for non-table blocks
#         if not block_info['is_table']:
#             if block_info['type'] == 0:  # Text block
#                 block_info['type'] = 'text'
#                 lines = block.get('lines', [])
#                 block_info['line_count'] = len(lines)
#                 block_info['word_count'] = 0
#                 block_info['font_sizes'] = set()
                
#                 # Analyze text content
#                 for line in lines:
#                     for span in line.get('spans', []):
#                         block_info['word_count'] += len(span.get('text', '').split())
#                         block_info['font_sizes'].add(round(span.get('size', 0), 1))
                
#                 block_info['font_sizes'] = list(block_info['font_sizes'])
                
#                 # Determine text block subtype
#                 if block_info['word_count'] > 100:
#                     block_info['subtype'] = 'paragraph'
#                 elif any(size > 14 for size in block_info['font_sizes']):
#                     block_info['subtype'] = 'heading'
#                 else:
#                     block_info['subtype'] = 'text_fragment'
                    
#             elif block_info['type'] == 1:  # Image block
#                 block_info['type'] = 'image'
#                 # Calculate approximate size
#                 bbox = block_info['bbox']
#                 width = bbox[2] - bbox[0]
#                 height = bbox[3] - bbox[1]
#                 block_info['subtype'] = 'large_image' if width * height > 10000 else 'small_image'
        
#         analyzed_blocks.append(block_info)
    
#     return analyzed_blocks, table_blocks

# def detect_tables_pymupdf(page):
#     """Detect tables using PyMuPDF's built-in methods"""
#     tables = []
    
#     try:
#         # Method 1: Use PyMuPDF's table finder (if available in your version)
#         # This is the most reliable method for table detection
#         if hasattr(page, 'find_tables'):
#             table_list = page.find_tables()
#             if table_list:
#                 for i, table in enumerate(table_list):
#                     tables.append({
#                         'id': f"pymupdf_{i}",
#                         'bbox': table.bbox,
#                         'rows': table.row_count,
#                         'cols': table.col_count,
#                         'method': 'pymupdf_builtin'
#                     })
#     except Exception as e:
#         print(f"PyMuPDF table detection error: {e}")
    
#     return tables

# def detect_tables_from_text(blocks):
#     """Detect tables by analyzing text structure and patterns"""
#     tables = []
#     table_candidates = []
    
#     # Look for table patterns in text blocks
#     text_blocks = [b for b in blocks['blocks'] if b.get('type') == 0]
    
#     # Group blocks that are aligned and have table-like structure
#     for i, block in enumerate(text_blocks):
#         lines = block.get('lines', [])
#         if len(lines) >= 3:  # Potential table with multiple rows
#             # Check for consistent alignment and spacing
#             if has_table_structure(block):
#                 table_candidates.append({
#                     'block': block,
#                     'bbox': block['bbox'],
#                     'confidence': 0.7
#                 })
    
#     # Merge overlapping table candidates
#     merged_tables = merge_table_candidates(table_candidates)
    
#     for i, table in enumerate(merged_tables):
#         tables.append({
#             'id': f"text_based_{i}",
#             'bbox': table['bbox'],
#             'rows': estimate_table_rows(table),
#             'cols': estimate_table_columns(table),
#             'method': 'text_analysis',
#             'confidence': table.get('confidence', 0.5)
#         })
    
#     return tables

# def has_table_structure(block):
#     """Check if a block has table-like structure"""
#     lines = block.get('lines', [])
#     if len(lines) < 2:
#         return False
    
#     # Check for consistent line spacing and alignment
#     spacings = []
#     for j in range(1, len(lines)):
#         prev_line_bbox = get_line_bbox(lines[j-1])
#         curr_line_bbox = get_line_bbox(lines[j])
#         spacing = curr_line_bbox[1] - prev_line_bbox[3]  # y0_current - y1_previous
#         spacings.append(spacing)
    
#     # If spacing is consistent, it might be a table
#     if len(spacings) > 1:
#         avg_spacing = sum(spacings) / len(spacings)
#         variance = sum((s - avg_spacing) ** 2 for s in spacings) / len(spacings)
#         if variance < 2.0:  # Low variance indicates consistent spacing
#             return True
    
#     return False

# def get_line_bbox(line):
#     """Get bounding box for a line"""
#     if not line.get('spans'):
#         return [0, 0, 0, 0]
    
#     x0 = min(span.get('bbox', [0, 0, 0, 0])[0] for span in line['spans'])
#     y0 = min(span.get('bbox', [0, 0, 0, 0])[1] for span in line['spans'])
#     x1 = max(span.get('bbox', [0, 0, 0, 0])[2] for span in line['spans'])
#     y1 = max(span.get('bbox', [0, 0, 0, 0])[3] for span in line['spans'])
    
#     return [x0, y0, x1, y1]

# def merge_table_candidates(candidates):
#     """Merge overlapping or adjacent table candidates"""
#     if not candidates:
#         return []
    
#     merged = []
#     candidates.sort(key=lambda x: x['bbox'][1])  # Sort by y-coordinate
    
#     current = candidates[0]
#     for candidate in candidates[1:]:
#         if bbox_overlap(current['bbox'], candidate['bbox']):
#             # Merge overlapping candidates
#             current['bbox'] = merge_bboxes([current['bbox'], candidate['bbox']])
#             current['confidence'] = max(current.get('confidence', 0.5), candidate.get('confidence', 0.5))
#         else:
#             merged.append(current)
#             current = candidate
    
#     merged.append(current)
#     return merged

# def bbox_overlap(bbox1, bbox2):
#     """Check if two bounding boxes overlap"""
#     return not (bbox1[2] < bbox2[0] or bbox1[0] > bbox2[2] or 
#                 bbox1[3] < bbox2[1] or bbox1[1] > bbox2[3])

# def merge_bboxes(bboxes):
#     """Merge multiple bounding boxes into one"""
#     if not bboxes:
#         return [0, 0, 0, 0]
    
#     x0 = min(bbox[0] for bbox in bboxes)
#     y0 = min(bbox[1] for bbox in bboxes)
#     x1 = max(bbox[2] for bbox in bboxes)
#     y1 = max(bbox[3] for bbox in bboxes)
    
#     return [x0, y0, x1, y1]

# def is_block_in_table(block_bbox, table_bbox, threshold=0.5):
#     """Check if a block is inside a table region"""
#     # Calculate intersection area
#     inter_x0 = max(block_bbox[0], table_bbox[0])
#     inter_y0 = max(block_bbox[1], table_bbox[1])
#     inter_x1 = min(block_bbox[2], table_bbox[2])
#     inter_y1 = min(block_bbox[3], table_bbox[3])
    
#     if inter_x0 >= inter_x1 or inter_y0 >= inter_y1:
#         return False
    
#     intersection_area = (inter_x1 - inter_x0) * (inter_y1 - inter_y0)
#     block_area = (block_bbox[2] - block_bbox[0]) * (block_bbox[3] - block_bbox[1])
    
#     # If more than threshold% of block is inside table, consider it part of table
#     return intersection_area / block_area > threshold

# def remove_duplicate_tables(tables, overlap_threshold=0.7):
#     """Remove duplicate tables that largely overlap"""
#     unique_tables = []
    
#     for table in tables:
#         is_duplicate = False
#         for existing in unique_tables:
#             if bbox_overlap_percentage(table['bbox'], existing['bbox']) > overlap_threshold:
#                 is_duplicate = True
#                 # Keep the one with higher confidence or more rows/cols
#                 if (table.get('rows', 0) * table.get('cols', 0) > 
#                     existing.get('rows', 0) * existing.get('cols', 0)):
#                     unique_tables.remove(existing)
#                     unique_tables.append(table)
#                 break
        
#         if not is_duplicate:
#             unique_tables.append(table)
    
#     return unique_tables

# def bbox_overlap_percentage(bbox1, bbox2):
#     """Calculate percentage overlap between two bounding boxes"""
#     inter_x0 = max(bbox1[0], bbox2[0])
#     inter_y0 = max(bbox1[1], bbox2[1])
#     inter_x1 = min(bbox1[2], bbox2[2])
#     inter_y1 = min(bbox1[3], bbox2[3])
    
#     if inter_x0 >= inter_x1 or inter_y0 >= inter_y1:
#         return 0.0
    
#     intersection_area = (inter_x1 - inter_x0) * (inter_y1 - inter_y0)
#     area1 = (bbox1[2] - bbox1[0]) * (bbox1[3] - bbox1[1])
#     area2 = (bbox2[2] - bbox2[0]) * (bbox2[3] - bbox2[1])
    
#     return intersection_area / min(area1, area2)

# def estimate_table_rows(table):
#     """Estimate number of rows in table"""
#     # This is a simplified estimation
#     return table.get('rows', 3)

# def estimate_table_columns(table):
#     """Estimate number of columns in table"""
#     # This is a simplified estimation
#     return table.get('cols', 2)

# def create_overlay_with_tables(page, analyzed_blocks, table_blocks, page_num, zoom_factor=2):
#     """Create transparent overlay with table highlighting"""
#     # Get page dimensions
#     mat = fitz.Matrix(zoom_factor, zoom_factor)
#     pix = page.get_pixmap(matrix=mat)
    
#     # Convert to PIL Image
#     img_data = pix.tobytes("png")
#     img = Image.open(io.BytesIO(img_data))
    
#     # Create transparent overlay
#     overlay = Image.new("RGBA", img.size, (0, 0, 0, 0))
#     draw = ImageDraw.Draw(overlay)
    
#     try:
#         font = ImageFont.truetype("arial.ttf", 12)
#         small_font = ImageFont.truetype("arial.ttf", 8)
#     except:
#         font = ImageFont.load_default()
#         small_font = ImageFont.load_default()
    
#     # Enhanced color scheme with table color
#     colors = {
#         'heading': (255, 100, 100, 96),      # Light red for headings
#         'paragraph': (100, 100, 255, 64),    # Light blue for paragraphs
#         'text_fragment': (100, 200, 100, 48), # Light green for text fragments
#         'large_image': (255, 200, 50, 96),   # Orange for large images
#         'small_image': (255, 150, 50, 64),   # Light orange for small images
#         'table': (180, 50, 230, 96),         # Purple for tables - NEW COLOR
#         'unknown': (150, 150, 150, 48)       # Gray for unknown
#     }
    
#     # First, draw table regions
#     for i, table in enumerate(table_blocks):
#         bbox = [coord * zoom_factor for coord in table['bbox']]
        
#         # Draw table region with purple color
#         draw.rectangle(bbox, fill=colors['table'], 
#                       outline=(colors['table'][0], colors['table'][1], colors['table'][2], 255), 
#                       width=3)
        
#         # Add table label
#         table_label = f"TABLE {i+1}"
#         if 'rows' in table and 'cols' in table:
#             table_label += f" ({table['rows']}x{table['cols']})"
        
#         text_bbox = draw.textbbox((bbox[0] + 5, bbox[1] + 5), table_label, font=font)
#         text_bg = [text_bbox[0]-3, text_bbox[1]-3, text_bbox[2]+3, text_bbox[3]+3]
#         draw.rectangle(text_bg, fill=(255, 255, 255, 224))
#         draw.text((bbox[0] + 5, bbox[1] + 5), table_label, fill=(0, 0, 0, 255), font=font)
    
#     # Then, draw other blocks
#     for block in analyzed_blocks:
#         if 'bbox' in block and not block.get('is_table', False):
#             # Scale bbox coordinates to match the high-res image
#             bbox = [coord * zoom_factor for coord in block['bbox']]
            
#             # Determine color based on subtype
#             color = colors.get(block['subtype'], colors['unknown'])
            
#             # Draw semi-transparent rectangle
#             draw.rectangle(bbox, fill=color, outline=(color[0], color[1], color[2], 255), width=2)
            
#             # Add block number
#             draw.text((bbox[0] + 5, bbox[1] + 5), str(block['block_number']), 
#                      fill=(0, 0, 0, 255), font=small_font)
    
#     # Add page info watermark with table count
#     table_count = len(table_blocks)
#     watermark_text = f"Page {page_num} - {len(analyzed_blocks)} blocks - {table_count} table(s)"
#     watermark_bbox = draw.textbbox((10, 10), watermark_text, font=small_font)
#     draw.rectangle([watermark_bbox[0]-5, watermark_bbox[1]-5, 
#                    watermark_bbox[2]+5, watermark_bbox[3]+5], 
#                    fill=(255, 255, 255, 192))
#     draw.text((10, 10), watermark_text, fill=(0, 0, 0, 255), font=small_font)
    
#     # Combine original image with overlay
#     result = Image.alpha_composite(img.convert("RGBA"), overlay)
#     return result

# def print_block_statistics_with_tables(analyzed_blocks, table_blocks, page_num):
#     """Print detailed statistics about blocks including tables"""
#     print(f"\nPage {page_num} Block Analysis:")
#     print(f"  Total blocks: {len(analyzed_blocks)}")
#     print(f"  Tables detected: {len(table_blocks)}")
    
#     type_count = {}
#     table_blocks_count = sum(1 for b in analyzed_blocks if b.get('is_table', False))
    
#     for block in analyzed_blocks:
#         block_type = block['type']
#         type_count[block_type] = type_count.get(block_type, 0) + 1
    
#     print("  Block types:")
#     for block_type, count in type_count.items():
#         print(f"    {block_type}: {count}")
    
#     if table_blocks_count > 0:
#         print(f"  Table blocks: {table_blocks_count}")
    
#     # Table details
#     for i, table in enumerate(table_blocks):
#         print(f"  Table {i+1}: {table['bbox']} "
#               f"({table.get('rows', '?')}x{table.get('cols', '?')}) "
#               f"- Method: {table.get('method', 'unknown')}")

# def find_tables_in_pdf(pdf_path):
#     """Quick function to find which pages have tables"""
#     doc = fitz.open(pdf_path)
#     table_pages = []
    
#     print(f"\n🔍 Scanning for tables in: {os.path.basename(pdf_path)}")
    
#     for page_num in range(len(doc)):
#         page = doc[page_num]
        
#         # Use PyMuPDF table detection
#         tables = detect_tables_pymupdf(page)
        
#         # Also check text-based tables
#         blocks = page.get_text("dict")
#         text_tables = detect_tables_from_text(blocks)
        
#         if tables or text_tables:
#             table_pages.append({
#                 'page': page_num,
#                 'pymupdf_tables': len(tables),
#                 'text_tables': len(text_tables),
#                 'total_tables': len(tables) + len(text_tables)
#             })
#             print(f"✅ Page {page_num}: {len(tables)} + {len(text_tables)} tables")
    
#     doc.close()
    
#     if table_pages:
#         print(f"\n🎯 TABLE SUMMARY: Found tables on {len(table_pages)} page(s)")
#         for page_info in table_pages:
#             print(f"   Page {page_info['page']}: {page_info['total_tables']} tables "
#                   f"(PyMuPDF: {page_info['pymupdf_tables']}, Text-based: {page_info['text_tables']})")
#     else:
#         print(f"\n❌ No tables found in the document")
    
#     return table_pages

# def main_enhanced():
#     """Enhanced main function with table detection focus"""
#     pdf_path = r"D:\Upwork\POC\RAG\Data\Financial results_bajaj.pdf"
    
#     # Check if file exists
#     if not os.path.exists(pdf_path):
#         print(f"Error: File not found - {pdf_path}")
#         return
    
#     # Display PDF information
#     display_pdf_info(pdf_path)
    
#     # Create output directory with timestamp
#     timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
#     output_dir = os.path.join(r"D:\Upwork\POC\RAG\Data\output", f"table_analysis_{timestamp}")
#     os.makedirs(output_dir, exist_ok=True)
    
#     print("\nChoose an option:")
#     print("1. Quick table scan (find which pages have tables)")
#     print("2. Full analysis with overlays")
#     print("3. Process specific page")
    
#     choice = input("Enter your choice (1-3): ").strip()
    
#     if choice == '1':
#         # Quick table scan
#         table_pages = find_tables_in_pdf(pdf_path)
        
#     elif choice == '2':
#         # Full analysis
#         output_base = os.path.join(output_dir, "financial_results_bajaj")
#         processed_data, table_pages = process_pdf_blocks_with_tables(
#             pdf_path, output_base, None, save_json=True)
        
#     elif choice == '3':
#         # Specific page
#         try:
#             page_num = int(input("Enter page number: "))
#             output_base = os.path.join(output_dir, f"financial_results_bajaj_page_{page_num}")
#             processed_data, table_pages = process_pdf_blocks_with_tables(
#                 pdf_path, output_base, page_num, save_json=True)
#         except ValueError:
#             print("Invalid page number!")
    
#     else:
#         print("Invalid choice!")

# def display_pdf_info(pdf_path):
#     """Display basic information about the PDF"""
#     try:
#         doc = fitz.open(pdf_path)
#         print(f"\nPDF Information:")
#         print(f"  File: {os.path.basename(pdf_path)}")
#         print(f"  Total pages: {len(doc)}")
#         print(f"  File size: {os.path.getsize(pdf_path) / 1024:.2f} KB")
#         doc.close()
#     except Exception as e:
#         print(f"Error reading PDF: {e}")

# if __name__ == "__main__":
#     main_enhanced()

In [56]:
# import fitz  # PyMuPDF
# import os
# import json
# from datetime import datetime
# import re

# def extract_text_from_pdf(pdf_path, output_dir, specific_page=None, ignore_images=True):
#     """Extract text from PDF while maintaining sequence and formatting tables as markdown"""
#     doc = fitz.open(pdf_path)
    
#     # Validate page number if specific page is requested
#     if specific_page is not None:
#         if specific_page < 0 or specific_page >= len(doc):
#             print(f"Error: Page {specific_page} is out of range. PDF has {len(doc)} pages.")
#             doc.close()
#             return
    
#     print(f"Extracting text from PDF: {os.path.basename(pdf_path)}")
#     print(f"Total pages: {len(doc)}")
    
#     if specific_page is not None:
#         pages_to_process = [specific_page]
#         print(f"Processing only page {specific_page}")
#     else:
#         pages_to_process = range(len(doc))
#         print("Processing all pages")
    
#     all_extracted_data = {}
    
#     for page_num in pages_to_process:
#         page = doc[page_num]
        
#         print(f"\n📄 Processing Page {page_num}...")
        
#         # Extract text with structure
#         extracted_content = extract_structured_text_from_page(page, page_num, ignore_images)
#         all_extracted_data[f"page_{page_num}"] = extracted_content
        
#         # Save individual page text
#         save_page_text(extracted_content, output_dir, page_num)
        
#         # Print summary
#         print(f"✅ Page {page_num} extracted:")
#         print(f"   - Text blocks: {len(extracted_content['text_blocks'])}")
#         print(f"   - Tables: {len(extracted_content['tables'])}")
#         print(f"   - Total characters: {len(extracted_content['full_text'])}")
    
#     # Save combined text
#     save_combined_text(all_extracted_data, output_dir)
    
#     doc.close()
#     return all_extracted_data

# def extract_structured_text_from_page(page, page_num, ignore_images=True):
#     """Extract structured text from a single page"""
#     # Get detailed text blocks as dictionary
#     blocks = page.get_text("dict")
    
#     # Detect tables first
#     tables = detect_tables_pymupdf(page)
#     text_based_tables = detect_tables_from_text(blocks)
#     all_tables = tables + text_based_tables
    
#     # Remove duplicate tables
#     unique_tables = remove_duplicate_tables(all_tables)
    
#     # Extract content in sequence
#     extracted_content = {
#         'page_number': page_num,
#         'text_blocks': [],
#         'tables': [],
#         'full_text': '',
#         'structured_content': []
#     }
    
#     # Process blocks in reading order (top to bottom, left to right)
#     sorted_blocks = sort_blocks_reading_order(blocks['blocks'])
    
#     for block in sorted_blocks:
#         block_bbox = block.get('bbox', [])
        
#         # Check if block is inside any table
#         is_in_table = False
#         for table in unique_tables:
#             if is_block_in_table(block_bbox, table['bbox']):
#                 is_in_table = True
#                 break
        
#         if is_in_table:
#             continue  # Skip table blocks here, we'll process tables separately
        
#         # Process non-table blocks
#         if block.get('type') == 0:  # Text block
#             text_content = extract_text_from_block(block)
#             if text_content.strip():
#                 extracted_content['text_blocks'].append({
#                     'bbox': block_bbox,
#                     'text': text_content,
#                     'type': 'text'
#                 })
#                 extracted_content['structured_content'].append({
#                     'type': 'text',
#                     'content': text_content,
#                     'bbox': block_bbox
#                 })
        
#         elif block.get('type') == 1 and not ignore_images:  # Image block
#             # You can add image processing here if needed
#             pass
    
#     # Process tables
#     for i, table in enumerate(unique_tables):
#         table_text = extract_table_as_markdown(page, table, page_num)
#         if table_text:
#             extracted_content['tables'].append({
#                 'table_id': i + 1,
#                 'bbox': table['bbox'],
#                 'markdown': table_text,
#                 'dimensions': f"{table.get('rows', '?')}x{table.get('cols', '?')}"
#             })
#             extracted_content['structured_content'].append({
#                 'type': 'table',
#                 'content': table_text,
#                 'bbox': table['bbox']
#             })
    
#     # Generate full text in correct sequence
#     full_text = generate_sequential_text(extracted_content['structured_content'])
#     extracted_content['full_text'] = full_text
    
#     return extracted_content

# def sort_blocks_reading_order(blocks):
#     """Sort blocks in reading order (top to bottom, left to right)"""
#     def block_sort_key(block):
#         bbox = block.get('bbox', [0, 0, 0, 0])
#         # Primary sort by Y (vertical position), secondary by X (horizontal position)
#         return (bbox[1], bbox[0])
    
#     return sorted(blocks, key=block_sort_key)

# def extract_text_from_block(block):
#     """Extract and clean text from a block"""
#     text_parts = []
    
#     if 'lines' in block:
#         for line in block['lines']:
#             line_text = ""
#             if 'spans' in line:
#                 for span in line['spans']:
#                     span_text = span.get('text', '').strip()
#                     if span_text:
#                         line_text += span_text + " "
            
#             line_text = line_text.strip()
#             if line_text:
#                 text_parts.append(line_text)
    
#     # Join lines with proper spacing
#     block_text = '\n'.join(text_parts)
    
#     # Clean up text
#     block_text = re.sub(r'\s+', ' ', block_text)  # Normalize whitespace
#     block_text = block_text.strip()
    
#     return block_text

# def extract_table_as_markdown(page, table, page_num):
#     """Extract table and format as markdown"""
#     try:
#         # Try to extract table using PyMuPDF's table extraction
#         if hasattr(page, 'find_tables') and table.get('method') == 'pymupdf_builtin':
#             tables = page.find_tables()
#             table_obj = tables[0] if tables else None
            
#             if table_obj and hasattr(table_obj, 'extract'):
#                 table_data = table_obj.extract()
#                 return format_table_as_markdown(table_data)
        
#         # Fallback: Extract text from table region and try to parse as table
#         table_bbox = table['bbox']
#         table_text = page.get_text("text", clip=table_bbox)
#         return parse_text_as_table(table_text, table_bbox)
        
#     except Exception as e:
#         print(f"Error extracting table from page {page_num}: {e}")
#         return f"<!-- Table extraction failed -->\n"

# def format_table_as_markdown(table_data):
#     """Format table data as markdown"""
#     if not table_data:
#         return "<!-- Empty table -->\n"
    
#     # Clean the table data
#     cleaned_data = []
#     for row in table_data:
#         cleaned_row = []
#         for cell in row:
#             if cell is None:
#                 cleaned_row.append("")
#             else:
#                 # Clean cell text
#                 cell_text = str(cell).strip()
#                 cell_text = re.sub(r'\s+', ' ', cell_text)
#                 cleaned_row.append(cell_text)
#         cleaned_data.append(cleaned_row)
    
#     # Remove empty rows
#     cleaned_data = [row for row in cleaned_data if any(cell.strip() for cell in row)]
    
#     if not cleaned_data:
#         return "<!-- Empty table after cleaning -->\n"
    
#     # Create markdown table
#     markdown_lines = []
    
#     # Add header (first row)
#     header_row = cleaned_data[0]
#     markdown_lines.append("| " + " | ".join(header_row) + " |")
    
#     # Add separator
#     separator = "| " + " | ".join(["---"] * len(header_row)) + " |"
#     markdown_lines.append(separator)
    
#     # Add data rows
#     for row in cleaned_data[1:]:
#         # Ensure row has same number of columns as header
#         while len(row) < len(header_row):
#             row.append("")
#         while len(row) > len(header_row):
#             row = row[:len(header_row)]
        
#         markdown_lines.append("| " + " | ".join(row) + " |")
    
#     return "\n".join(markdown_lines) + "\n\n"

# def parse_text_as_table(table_text, bbox):
#     """Parse raw text as table when structured extraction fails"""
#     lines = table_text.strip().split('\n')
#     if not lines:
#         return "<!-- No table content -->\n"
    
#     # Simple table detection based on consistent columns
#     potential_table_data = []
    
#     for line in lines:
#         line = line.strip()
#         if not line:
#             continue
        
#         # Split by multiple spaces or tabs
#         cells = re.split(r'\s{2,}|\t', line)
#         cells = [cell.strip() for cell in cells if cell.strip()]
        
#         if len(cells) >= 2:  # Likely a table row
#             potential_table_data.append(cells)
    
#     if len(potential_table_data) >= 2:
#         return format_table_as_markdown(potential_table_data)
#     else:
#         # If it doesn't look like a table, return as preformatted text
#         return "```\n" + table_text + "\n```\n\n"

# def generate_sequential_text(structured_content):
#     """Generate sequential text from structured content"""
#     # Sort content by vertical position
#     sorted_content = sorted(structured_content, key=lambda x: x['bbox'][1])
    
#     full_text_parts = []
#     current_y = -1
#     y_threshold = 10  # pixels threshold for new line
    
#     for item in sorted_content:
#         item_y = item['bbox'][1]
        
#         # Add newline if significant vertical gap
#         if current_y != -1 and item_y > current_y + y_threshold:
#             full_text_parts.append('\n')
        
#         if item['type'] == 'text':
#             full_text_parts.append(item['content'] + '\n')
#         elif item['type'] == 'table':
#             full_text_parts.append('\n' + item['content'] + '\n')
        
#         current_y = item['bbox'][3]  # Bottom of current item
    
#     return ''.join(full_text_parts)

# def save_page_text(extracted_content, output_dir, page_num):
#     """Save extracted text for a single page"""
#     filename = f"page_{page_num:03d}_extracted.txt"
#     filepath = os.path.join(output_dir, filename)
    
#     with open(filepath, 'w', encoding='utf-8') as f:
#         f.write(f"=== PAGE {page_num} EXTRACTED TEXT ===\n\n")
#         f.write(extracted_content['full_text'])
    
#     print(f"📁 Saved: {filename}")

# def save_combined_text(all_extracted_data, output_dir):
#     """Save combined text from all pages"""
#     filename = "combined_extracted_text.txt"
#     filepath = os.path.join(output_dir, filename)
    
#     with open(filepath, 'w', encoding='utf-8') as f:
#         f.write("=== COMBINED EXTRACTED TEXT ===\n\n")
        
#         for page_key in sorted(all_extracted_data.keys()):
#             page_data = all_extracted_data[page_key]
#             f.write(f"\n{'='*50}\n")
#             f.write(f"PAGE {page_data['page_number']}\n")
#             f.write(f"{'='*50}\n\n")
#             f.write(page_data['full_text'])
#             f.write("\n\n")
    
#     print(f"📁 Saved combined text: {filename}")

# # FIXED table detection functions
# def detect_tables_pymupdf(page):
#     """Detect tables using PyMuPDF's built-in methods"""
#     tables = []
    
#     try:
#         if hasattr(page, 'find_tables'):
#             table_list = page.find_tables()
#             if table_list:
#                 for i, table in enumerate(table_list):
#                     tables.append({
#                         'id': f"pymupdf_{i}",
#                         'bbox': table.bbox,
#                         'rows': table.row_count,
#                         'cols': table.col_count,
#                         'method': 'pymupdf_builtin'
#                     })
#     except Exception as e:
#         print(f"PyMuPDF table detection error: {e}")
    
#     return tables

# def detect_tables_from_text(blocks_dict):
#     """Detect tables by analyzing text structure and patterns - FIXED VERSION"""
#     tables = []
#     table_candidates = []
    
#     # FIX: Access the blocks list from the dictionary
#     if 'blocks' in blocks_dict:
#         text_blocks = [b for b in blocks_dict['blocks'] if b.get('type') == 0]
#     else:
#         # If it's already a list of blocks
#         text_blocks = [b for b in blocks_dict if b.get('type') == 0]
    
#     for i, block in enumerate(text_blocks):
#         lines = block.get('lines', [])
#         if len(lines) >= 3:
#             if has_table_structure(block):
#                 table_candidates.append({
#                     'block': block,
#                     'bbox': block['bbox'],
#                     'confidence': 0.7
#                 })
    
#     merged_tables = merge_table_candidates(table_candidates)
    
#     for i, table in enumerate(merged_tables):
#         tables.append({
#             'id': f"text_based_{i}",
#             'bbox': table['bbox'],
#             'rows': estimate_table_rows(table),
#             'cols': estimate_table_columns(table),
#             'method': 'text_analysis',
#             'confidence': table.get('confidence', 0.5)
#         })
    
#     return tables

# def has_table_structure(block):
#     """Check if a block has table-like structure"""
#     lines = block.get('lines', [])
#     if len(lines) < 2:
#         return False
    
#     spacings = []
#     for j in range(1, len(lines)):
#         prev_line_bbox = get_line_bbox(lines[j-1])
#         curr_line_bbox = get_line_bbox(lines[j])
#         spacing = curr_line_bbox[1] - prev_line_bbox[3]
#         spacings.append(spacing)
    
#     if len(spacings) > 1:
#         avg_spacing = sum(spacings) / len(spacings)
#         variance = sum((s - avg_spacing) ** 2 for s in spacings) / len(spacings)
#         if variance < 2.0:
#             return True
    
#     return False

# def get_line_bbox(line):
#     """Get bounding box for a line"""
#     if not line.get('spans'):
#         return [0, 0, 0, 0]
    
#     x0 = min(span.get('bbox', [0, 0, 0, 0])[0] for span in line['spans'])
#     y0 = min(span.get('bbox', [0, 0, 0, 0])[1] for span in line['spans'])
#     x1 = max(span.get('bbox', [0, 0, 0, 0])[2] for span in line['spans'])
#     y1 = max(span.get('bbox', [0, 0, 0, 0])[3] for span in line['spans'])
    
#     return [x0, y0, x1, y1]

# def merge_table_candidates(candidates):
#     """Merge overlapping or adjacent table candidates"""
#     if not candidates:
#         return []
    
#     merged = []
#     candidates.sort(key=lambda x: x['bbox'][1])
    
#     current = candidates[0]
#     for candidate in candidates[1:]:
#         if bbox_overlap(current['bbox'], candidate['bbox']):
#             current['bbox'] = merge_bboxes([current['bbox'], candidate['bbox']])
#             current['confidence'] = max(current.get('confidence', 0.5), candidate.get('confidence', 0.5))
#         else:
#             merged.append(current)
#             current = candidate
    
#     merged.append(current)
#     return merged

# def bbox_overlap(bbox1, bbox2):
#     """Check if two bounding boxes overlap"""
#     return not (bbox1[2] < bbox2[0] or bbox1[0] > bbox2[2] or 
#                 bbox1[3] < bbox2[1] or bbox1[1] > bbox2[3])

# def merge_bboxes(bboxes):
#     """Merge multiple bounding boxes into one"""
#     if not bboxes:
#         return [0, 0, 0, 0]
    
#     x0 = min(bbox[0] for bbox in bboxes)
#     y0 = min(bbox[1] for bbox in bboxes)
#     x1 = max(bbox[2] for bbox in bboxes)
#     y1 = max(bbox[3] for bbox in bboxes)
    
#     return [x0, y0, x1, y1]

# def is_block_in_table(block_bbox, table_bbox, threshold=0.5):
#     """Check if a block is inside a table region"""
#     inter_x0 = max(block_bbox[0], table_bbox[0])
#     inter_y0 = max(block_bbox[1], table_bbox[1])
#     inter_x1 = min(block_bbox[2], table_bbox[2])
#     inter_y1 = min(block_bbox[3], table_bbox[3])
    
#     if inter_x0 >= inter_x1 or inter_y0 >= inter_y1:
#         return False
    
#     intersection_area = (inter_x1 - inter_x0) * (inter_y1 - inter_y0)
#     block_area = (block_bbox[2] - block_bbox[0]) * (block_bbox[3] - block_bbox[1])
    
#     return intersection_area / block_area > threshold

# def remove_duplicate_tables(tables, overlap_threshold=0.7):
#     """Remove duplicate tables that largely overlap"""
#     unique_tables = []
    
#     for table in tables:
#         is_duplicate = False
#         for existing in unique_tables:
#             if bbox_overlap_percentage(table['bbox'], existing['bbox']) > overlap_threshold:
#                 is_duplicate = True
#                 if (table.get('rows', 0) * table.get('cols', 0) > 
#                     existing.get('rows', 0) * existing.get('cols', 0)):
#                     unique_tables.remove(existing)
#                     unique_tables.append(table)
#                 break
        
#         if not is_duplicate:
#             unique_tables.append(table)
    
#     return unique_tables

# def bbox_overlap_percentage(bbox1, bbox2):
#     """Calculate percentage overlap between two bounding boxes"""
#     inter_x0 = max(bbox1[0], bbox2[0])
#     inter_y0 = max(bbox1[1], bbox2[1])
#     inter_x1 = min(bbox1[2], bbox2[2])
#     inter_y1 = min(bbox1[3], bbox2[3])
    
#     if inter_x0 >= inter_x1 or inter_y0 >= inter_y1:
#         return 0.0
    
#     intersection_area = (inter_x1 - inter_x0) * (inter_y1 - inter_y0)
#     area1 = (bbox1[2] - bbox1[0]) * (bbox1[3] - bbox1[1])
#     area2 = (bbox2[2] - bbox2[0]) * (bbox2[3] - bbox2[1])
    
#     return intersection_area / min(area1, area2)

# def estimate_table_rows(table):
#     """Estimate number of rows in table"""
#     return table.get('rows', 3)

# def estimate_table_columns(table):
#     """Estimate number of columns in table"""
#     return table.get('cols', 2)

# def main_text_extraction():
#     """Main function for text extraction"""
#     pdf_path = r"D:\Upwork\POC\RAG\Data\Financial results_bajaj.pdf"
    
#     if not os.path.exists(pdf_path):
#         print(f"Error: File not found - {pdf_path}")
#         return
    
#     # Create output directory
#     timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
#     output_dir = os.path.join(r"D:\Upwork\POC\RAG\Data\output", f"text_extraction_{timestamp}")
#     os.makedirs(output_dir, exist_ok=True)
    
#     print("Text Extraction Options:")
#     print("1. Extract all pages")
#     print("2. Extract specific page")
#     print("3. Extract page range")
    
#     choice = input("Enter your choice (1-3): ").strip()
    
#     if choice == '1':
#         extracted_data = extract_text_from_pdf(pdf_path, output_dir)
        
#     elif choice == '2':
#         try:
#             page_num = int(input("Enter page number: "))
#             extracted_data = extract_text_from_pdf(pdf_path, output_dir, specific_page=page_num)
#         except ValueError:
#             print("Invalid page number!")
            
#     elif choice == '3':
#         try:
#             start_page = int(input("Enter start page: "))
#             end_page = int(input("Enter end page: "))
            
#             # Process page range
#             doc = fitz.open(pdf_path)
#             for page_num in range(start_page, end_page + 1):
#                 if 0 <= page_num < len(doc):
#                     extract_text_from_pdf(pdf_path, output_dir, specific_page=page_num)
#             doc.close()
            
#         except ValueError:
#             print("Invalid page numbers!")
    
#     else:
#         print("Invalid choice!")

# # Simple function for quick text extraction without tables
# def extract_simple_text(pdf_path, output_dir, specific_page=None):
#     """Simple text extraction without complex table processing"""
#     doc = fitz.open(pdf_path)
    
#     if specific_page is not None:
#         pages_to_process = [specific_page]
#     else:
#         pages_to_process = range(len(doc))
    
#     for page_num in pages_to_process:
#         page = doc[page_num]
        
#         # Extract simple text
#         text = page.get_text("text")
        
#         # Clean the text
#         text = re.sub(r'\n\s*\n', '\n\n', text)  # Normalize line breaks
#         text = text.strip()
        
#         # Save to file
#         filename = f"page_{page_num:03d}_simple.txt"
#         filepath = os.path.join(output_dir, filename)
        
#         with open(filepath, 'w', encoding='utf-8') as f:
#             f.write(f"=== PAGE {page_num} ===\n\n")
#             f.write(text)
        
#         print(f"Saved: {filename} - {len(text)} characters")
    
#     doc.close()

# if __name__ == "__main__":
#     main_text_extraction()

In [58]:
# import fitz  # PyMuPDF
# import os
# import json
# from datetime import datetime
# import re
# from PIL import Image, ImageDraw, ImageFont
# import io
# import concurrent.futures
# import threading

# def process_pdf_parallel(pdf_path, output_dir, specific_page=None, ignore_images=True):
#     """Process PDF in parallel - extract text and create overlays simultaneously"""
#     doc = fitz.open(pdf_path)
    
#     # Validate page number if specific page is requested
#     if specific_page is not None:
#         if specific_page < 0 or specific_page >= len(doc):
#             print(f"Error: Page {specific_page} is out of range. PDF has {len(doc)} pages.")
#             doc.close()
#             return
    
#     print(f"Parallel Processing PDF: {os.path.basename(pdf_path)}")
#     print(f"Total pages: {len(doc)}")
    
#     if specific_page is not None:
#         pages_to_process = [specific_page]
#         print(f"Processing only page {specific_page}")
#     else:
#         pages_to_process = range(len(doc))
#         print("Processing all pages")
    
#     # Create subdirectories for organized output
#     text_dir = os.path.join(output_dir, "text_extraction")
#     overlay_dir = os.path.join(output_dir, "overlay_images")
#     os.makedirs(text_dir, exist_ok=True)
#     os.makedirs(overlay_dir, exist_ok=True)
    
#     all_extracted_data = {}
    
#     # Process pages in parallel
#     with concurrent.futures.ThreadPoolExecutor(max_workers=4) as executor:
#         future_to_page = {}
        
#         for page_num in pages_to_process:
#             page = doc[page_num]
            
#             # Submit both text extraction and overlay creation tasks
#             future = executor.submit(process_single_page_parallel, 
#                                    page, page_num, text_dir, overlay_dir, ignore_images)
#             future_to_page[future] = page_num
        
#         # Collect results
#         for future in concurrent.futures.as_completed(future_to_page):
#             page_num = future_to_page[future]
#             try:
#                 page_data, overlay_path = future.result()
#                 all_extracted_data[f"page_{page_num}"] = page_data
#                 print(f"✅ Page {page_num} completed - Text: {len(page_data['full_text'])} chars, Overlay: {overlay_path}")
#             except Exception as e:
#                 print(f"❌ Page {page_num} failed: {e}")
    
#     # Save combined text
#     save_combined_text(all_extracted_data, text_dir)
    
#     doc.close()
#     return all_extracted_data

# def process_single_page_parallel(page, page_num, text_dir, overlay_dir, ignore_images=True):
#     """Process a single page for both text extraction and overlay creation"""
#     # Extract structured text
#     extracted_content = extract_structured_text_from_page(page, page_num, ignore_images)
    
#     # Create overlay image
#     overlay_path = create_page_overlay(page, page_num, overlay_dir, extracted_content)
    
#     # Save individual page text
#     save_page_text(extracted_content, text_dir, page_num)
    
#     return extracted_content, overlay_path

# def create_page_overlay(page, page_num, overlay_dir, extracted_content=None, zoom_factor=2):
#     """Create overlay image for a single page"""
#     # Get page dimensions
#     mat = fitz.Matrix(zoom_factor, zoom_factor)
#     pix = page.get_pixmap(matrix=mat)
    
#     # Convert to PIL Image
#     img_data = pix.tobytes("png")
#     img = Image.open(io.BytesIO(img_data))
    
#     # Create transparent overlay
#     overlay = Image.new("RGBA", img.size, (0, 0, 0, 0))
#     draw = ImageDraw.Draw(overlay)
    
#     try:
#         font = ImageFont.truetype("arial.ttf", 12)
#         small_font = ImageFont.truetype("arial.ttf", 8)
#     except:
#         font = ImageFont.load_default()
#         small_font = ImageFont.load_default()
    
#     # Color scheme
#     colors = {
#         'heading': (255, 100, 100, 96),
#         'paragraph': (100, 100, 255, 64),
#         'text_fragment': (100, 200, 100, 48),
#         'table': (180, 50, 230, 96),  # Purple for tables
#         'unknown': (150, 150, 150, 48)
#     }
    
#     # Draw blocks if we have extracted content
#     if extracted_content and 'structured_content' in extracted_content:
#         for item in extracted_content['structured_content']:
#             bbox = [coord * zoom_factor for coord in item['bbox']]
            
#             if item['type'] == 'table':
#                 color = colors['table']
#                 # Draw table with thicker border
#                 draw.rectangle(bbox, fill=color, 
#                               outline=(color[0], color[1], color[2], 255), 
#                               width=3)
                
#                 # Add table label
#                 table_label = "TABLE"
#                 text_bbox = draw.textbbox((bbox[0] + 5, bbox[1] + 5), table_label, font=font)
#                 text_bg = [text_bbox[0]-3, text_bbox[1]-3, text_bbox[2]+3, text_bbox[3]+3]
#                 draw.rectangle(text_bg, fill=(255, 255, 255, 224))
#                 draw.text((bbox[0] + 5, bbox[1] + 5), table_label, fill=(0, 0, 0, 255), font=font)
            
#             else:  # text blocks
#                 color = colors.get(item.get('subtype', 'unknown'), colors['unknown'])
#                 draw.rectangle(bbox, fill=color, 
#                               outline=(color[0], color[1], color[2], 255), 
#                               width=2)
    
#     # Add page info watermark
#     watermark_text = f"Page {page_num}"
#     if extracted_content:
#         block_count = len(extracted_content.get('text_blocks', []))
#         table_count = len(extracted_content.get('tables', []))
#         watermark_text += f" - {block_count} blocks, {table_count} tables"
    
#     watermark_bbox = draw.textbbox((10, 10), watermark_text, font=small_font)
#     draw.rectangle([watermark_bbox[0]-5, watermark_bbox[1]-5, 
#                    watermark_bbox[2]+5, watermark_bbox[3]+5], 
#                    fill=(255, 255, 255, 192))
#     draw.text((10, 10), watermark_text, fill=(0, 0, 0, 255), font=small_font)
    
#     # Combine original image with overlay
#     result = Image.alpha_composite(img.convert("RGBA"), overlay)
    
#     # Save overlay image
#     overlay_filename = f"page_{page_num:03d}_overlay.png"
#     overlay_path = os.path.join(overlay_dir, overlay_filename)
#     result.save(overlay_path, "PNG")
    
#     return overlay_path

# # Reuse the text extraction functions from previous implementation
# def extract_structured_text_from_page(page, page_num, ignore_images=True):
#     """Extract structured text from a single page"""
#     # Get detailed text blocks as dictionary
#     blocks = page.get_text("dict")
    
#     # Detect tables first
#     tables = detect_tables_pymupdf(page)
#     text_based_tables = detect_tables_from_text(blocks)
#     all_tables = tables + text_based_tables
    
#     # Remove duplicate tables
#     unique_tables = remove_duplicate_tables(all_tables)
    
#     # Extract content in sequence
#     extracted_content = {
#         'page_number': page_num,
#         'text_blocks': [],
#         'tables': [],
#         'full_text': '',
#         'structured_content': []
#     }
    
#     # Process blocks in reading order (top to bottom, left to right)
#     sorted_blocks = sort_blocks_reading_order(blocks['blocks'])
    
#     for block in sorted_blocks:
#         block_bbox = block.get('bbox', [])
        
#         # Check if block is inside any table
#         is_in_table = False
#         for table in unique_tables:
#             if is_block_in_table(block_bbox, table['bbox']):
#                 is_in_table = True
#                 break
        
#         if is_in_table:
#             continue  # Skip table blocks here, we'll process tables separately
        
#         # Process non-table blocks
#         if block.get('type') == 0:  # Text block
#             text_content = extract_text_from_block(block)
#             if text_content.strip():
#                 # Determine block subtype
#                 subtype = classify_text_block(block, text_content)
                
#                 extracted_content['text_blocks'].append({
#                     'bbox': block_bbox,
#                     'text': text_content,
#                     'type': 'text',
#                     'subtype': subtype
#                 })
#                 extracted_content['structured_content'].append({
#                     'type': 'text',
#                     'content': text_content,
#                     'bbox': block_bbox,
#                     'subtype': subtype
#                 })
        
#         elif block.get('type') == 1 and not ignore_images:  # Image block
#             # You can add image processing here if needed
#             pass
    
#     # Process tables
#     for i, table in enumerate(unique_tables):
#         table_text = extract_table_as_markdown(page, table, page_num)
#         if table_text:
#             extracted_content['tables'].append({
#                 'table_id': i + 1,
#                 'bbox': table['bbox'],
#                 'markdown': table_text,
#                 'dimensions': f"{table.get('rows', '?')}x{table.get('cols', '?')}"
#             })
#             extracted_content['structured_content'].append({
#                 'type': 'table',
#                 'content': table_text,
#                 'bbox': table['bbox']
#             })
    
#     # Generate full text in correct sequence
#     full_text = generate_sequential_text(extracted_content['structured_content'])
#     extracted_content['full_text'] = full_text
    
#     return extracted_content

# def classify_text_block(block, text_content):
#     """Classify text block into subtypes"""
#     lines = block.get('lines', [])
#     word_count = len(text_content.split())
    
#     # Check for large fonts (headings)
#     max_font_size = 0
#     for line in lines:
#         for span in line.get('spans', []):
#             max_font_size = max(max_font_size, span.get('size', 0))
    
#     if max_font_size > 14:
#         return 'heading'
#     elif word_count > 100:
#         return 'paragraph'
#     else:
#         return 'text_fragment'

# def sort_blocks_reading_order(blocks):
#     """Sort blocks in reading order (top to bottom, left to right)"""
#     def block_sort_key(block):
#         bbox = block.get('bbox', [0, 0, 0, 0])
#         return (bbox[1], bbox[0])
    
#     return sorted(blocks, key=block_sort_key)

# def extract_text_from_block(block):
#     """Extract and clean text from a block"""
#     text_parts = []
    
#     if 'lines' in block:
#         for line in block['lines']:
#             line_text = ""
#             if 'spans' in line:
#                 for span in line['spans']:
#                     span_text = span.get('text', '').strip()
#                     if span_text:
#                         line_text += span_text + " "
            
#             line_text = line_text.strip()
#             if line_text:
#                 text_parts.append(line_text)
    
#     # Join lines with proper spacing
#     block_text = '\n'.join(text_parts)
    
#     # Clean up text
#     block_text = re.sub(r'\s+', ' ', block_text)
#     block_text = block_text.strip()
    
#     return block_text

# def extract_table_as_markdown(page, table, page_num):
#     """Extract table and format as markdown"""
#     try:
#         # Try to extract table using PyMuPDF's table extraction
#         if hasattr(page, 'find_tables') and table.get('method') == 'pymupdf_builtin':
#             tables = page.find_tables()
#             table_obj = tables[0] if tables else None
            
#             if table_obj and hasattr(table_obj, 'extract'):
#                 table_data = table_obj.extract()
#                 return format_table_as_markdown(table_data)
        
#         # Fallback: Extract text from table region and try to parse as table
#         table_bbox = table['bbox']
#         table_text = page.get_text("text", clip=table_bbox)
#         return parse_text_as_table(table_text, table_bbox)
        
#     except Exception as e:
#         print(f"Error extracting table from page {page_num}: {e}")
#         return f"<!-- Table extraction failed -->\n"

# def format_table_as_markdown(table_data):
#     """Format table data as markdown"""
#     if not table_data:
#         return "<!-- Empty table -->\n"
    
#     # Clean the table data
#     cleaned_data = []
#     for row in table_data:
#         cleaned_row = []
#         for cell in row:
#             if cell is None:
#                 cleaned_row.append("")
#             else:
#                 cell_text = str(cell).strip()
#                 cell_text = re.sub(r'\s+', ' ', cell_text)
#                 cleaned_row.append(cell_text)
#         cleaned_data.append(cleaned_row)
    
#     # Remove empty rows
#     cleaned_data = [row for row in cleaned_data if any(cell.strip() for cell in row)]
    
#     if not cleaned_data:
#         return "<!-- Empty table after cleaning -->\n"
    
#     # Create markdown table
#     markdown_lines = []
    
#     # Add header (first row)
#     header_row = cleaned_data[0]
#     markdown_lines.append("| " + " | ".join(header_row) + " |")
    
#     # Add separator
#     separator = "| " + " | ".join(["---"] * len(header_row)) + " |"
#     markdown_lines.append(separator)
    
#     # Add data rows
#     for row in cleaned_data[1:]:
#         while len(row) < len(header_row):
#             row.append("")
#         while len(row) > len(header_row):
#             row = row[:len(header_row)]
        
#         markdown_lines.append("| " + " | ".join(row) + " |")
    
#     return "\n".join(markdown_lines) + "\n\n"

# def parse_text_as_table(table_text, bbox):
#     """Parse raw text as table when structured extraction fails"""
#     lines = table_text.strip().split('\n')
#     if not lines:
#         return "<!-- No table content -->\n"
    
#     potential_table_data = []
    
#     for line in lines:
#         line = line.strip()
#         if not line:
#             continue
        
#         cells = re.split(r'\s{2,}|\t', line)
#         cells = [cell.strip() for cell in cells if cell.strip()]
        
#         if len(cells) >= 2:
#             potential_table_data.append(cells)
    
#     if len(potential_table_data) >= 2:
#         return format_table_as_markdown(potential_table_data)
#     else:
#         return "```\n" + table_text + "\n```\n\n"

# def generate_sequential_text(structured_content):
#     """Generate sequential text from structured content"""
#     sorted_content = sorted(structured_content, key=lambda x: x['bbox'][1])
    
#     full_text_parts = []
#     current_y = -1
#     y_threshold = 10
    
#     for item in sorted_content:
#         item_y = item['bbox'][1]
        
#         if current_y != -1 and item_y > current_y + y_threshold:
#             full_text_parts.append('\n')
        
#         if item['type'] == 'text':
#             full_text_parts.append(item['content'] + '\n')
#         elif item['type'] == 'table':
#             full_text_parts.append('\n' + item['content'] + '\n')
        
#         current_y = item['bbox'][3]
    
#     return ''.join(full_text_parts)

# def save_page_text(extracted_content, output_dir, page_num):
#     """Save extracted text for a single page"""
#     filename = f"page_{page_num:03d}_extracted.txt"
#     filepath = os.path.join(output_dir, filename)
    
#     with open(filepath, 'w', encoding='utf-8') as f:
#         f.write(f"=== PAGE {page_num} EXTRACTED TEXT ===\n\n")
#         f.write(extracted_content['full_text'])
    
#     print(f"📄 Saved text: {filename}")

# def save_combined_text(all_extracted_data, output_dir):
#     """Save combined text from all pages"""
#     filename = "combined_extracted_text.txt"
#     filepath = os.path.join(output_dir, filename)
    
#     with open(filepath, 'w', encoding='utf-8') as f:
#         f.write("=== COMBINED EXTRACTED TEXT ===\n\n")
        
#         for page_key in sorted(all_extracted_data.keys()):
#             page_data = all_extracted_data[page_key]
#             f.write(f"\n{'='*50}\n")
#             f.write(f"PAGE {page_data['page_number']}\n")
#             f.write(f"{'='*50}\n\n")
#             f.write(page_data['full_text'])
#             f.write("\n\n")
    
#     print(f"📚 Saved combined text: {filename}")

# # Table detection functions (reused from previous implementation)
# def detect_tables_pymupdf(page):
#     """Detect tables using PyMuPDF's built-in methods"""
#     tables = []
    
#     try:
#         if hasattr(page, 'find_tables'):
#             table_list = page.find_tables()
#             if table_list:
#                 for i, table in enumerate(table_list):
#                     tables.append({
#                         'id': f"pymupdf_{i}",
#                         'bbox': table.bbox,
#                         'rows': table.row_count,
#                         'cols': table.col_count,
#                         'method': 'pymupdf_builtin'
#                     })
#     except Exception as e:
#         print(f"PyMuPDF table detection error: {e}")
    
#     return tables

# def detect_tables_from_text(blocks_dict):
#     """Detect tables by analyzing text structure and patterns"""
#     tables = []
#     table_candidates = []
    
#     if 'blocks' in blocks_dict:
#         text_blocks = [b for b in blocks_dict['blocks'] if b.get('type') == 0]
#     else:
#         text_blocks = [b for b in blocks_dict if b.get('type') == 0]
    
#     for i, block in enumerate(text_blocks):
#         lines = block.get('lines', [])
#         if len(lines) >= 3:
#             if has_table_structure(block):
#                 table_candidates.append({
#                     'block': block,
#                     'bbox': block['bbox'],
#                     'confidence': 0.7
#                 })
    
#     merged_tables = merge_table_candidates(table_candidates)
    
#     for i, table in enumerate(merged_tables):
#         tables.append({
#             'id': f"text_based_{i}",
#             'bbox': table['bbox'],
#             'rows': estimate_table_rows(table),
#             'cols': estimate_table_columns(table),
#             'method': 'text_analysis',
#             'confidence': table.get('confidence', 0.5)
#         })
    
#     return tables

# def has_table_structure(block):
#     """Check if a block has table-like structure"""
#     lines = block.get('lines', [])
#     if len(lines) < 2:
#         return False
    
#     spacings = []
#     for j in range(1, len(lines)):
#         prev_line_bbox = get_line_bbox(lines[j-1])
#         curr_line_bbox = get_line_bbox(lines[j])
#         spacing = curr_line_bbox[1] - prev_line_bbox[3]
#         spacings.append(spacing)
    
#     if len(spacings) > 1:
#         avg_spacing = sum(spacings) / len(spacings)
#         variance = sum((s - avg_spacing) ** 2 for s in spacings) / len(spacings)
#         if variance < 2.0:
#             return True
    
#     return False

# def get_line_bbox(line):
#     """Get bounding box for a line"""
#     if not line.get('spans'):
#         return [0, 0, 0, 0]
    
#     x0 = min(span.get('bbox', [0, 0, 0, 0])[0] for span in line['spans'])
#     y0 = min(span.get('bbox', [0, 0, 0, 0])[1] for span in line['spans'])
#     x1 = max(span.get('bbox', [0, 0, 0, 0])[2] for span in line['spans'])
#     y1 = max(span.get('bbox', [0, 0, 0, 0])[3] for span in line['spans'])
    
#     return [x0, y0, x1, y1]

# def merge_table_candidates(candidates):
#     """Merge overlapping or adjacent table candidates"""
#     if not candidates:
#         return []
    
#     merged = []
#     candidates.sort(key=lambda x: x['bbox'][1])
    
#     current = candidates[0]
#     for candidate in candidates[1:]:
#         if bbox_overlap(current['bbox'], candidate['bbox']):
#             current['bbox'] = merge_bboxes([current['bbox'], candidate['bbox']])
#             current['confidence'] = max(current.get('confidence', 0.5), candidate.get('confidence', 0.5))
#         else:
#             merged.append(current)
#             current = candidate
    
#     merged.append(current)
#     return merged

# def bbox_overlap(bbox1, bbox2):
#     """Check if two bounding boxes overlap"""
#     return not (bbox1[2] < bbox2[0] or bbox1[0] > bbox2[2] or 
#                 bbox1[3] < bbox2[1] or bbox1[1] > bbox2[3])

# def merge_bboxes(bboxes):
#     """Merge multiple bounding boxes into one"""
#     if not bboxes:
#         return [0, 0, 0, 0]
    
#     x0 = min(bbox[0] for bbox in bboxes)
#     y0 = min(bbox[1] for bbox in bboxes)
#     x1 = max(bbox[2] for bbox in bboxes)
#     y1 = max(bbox[3] for bbox in bboxes)
    
#     return [x0, y0, x1, y1]

# def is_block_in_table(block_bbox, table_bbox, threshold=0.5):
#     """Check if a block is inside a table region"""
#     inter_x0 = max(block_bbox[0], table_bbox[0])
#     inter_y0 = max(block_bbox[1], table_bbox[1])
#     inter_x1 = min(block_bbox[2], table_bbox[2])
#     inter_y1 = min(block_bbox[3], table_bbox[3])
    
#     if inter_x0 >= inter_x1 or inter_y0 >= inter_y1:
#         return False
    
#     intersection_area = (inter_x1 - inter_x0) * (inter_y1 - inter_y0)
#     block_area = (block_bbox[2] - block_bbox[0]) * (block_bbox[3] - block_bbox[1])
    
#     return intersection_area / block_area > threshold

# def remove_duplicate_tables(tables, overlap_threshold=0.7):
#     """Remove duplicate tables that largely overlap"""
#     unique_tables = []
    
#     for table in tables:
#         is_duplicate = False
#         for existing in unique_tables:
#             if bbox_overlap_percentage(table['bbox'], existing['bbox']) > overlap_threshold:
#                 is_duplicate = True
#                 if (table.get('rows', 0) * table.get('cols', 0) > 
#                     existing.get('rows', 0) * existing.get('cols', 0)):
#                     unique_tables.remove(existing)
#                     unique_tables.append(table)
#                 break
        
#         if not is_duplicate:
#             unique_tables.append(table)
    
#     return unique_tables

# def bbox_overlap_percentage(bbox1, bbox2):
#     """Calculate percentage overlap between two bounding boxes"""
#     inter_x0 = max(bbox1[0], bbox2[0])
#     inter_y0 = max(bbox1[1], bbox2[1])
#     inter_x1 = min(bbox1[2], bbox2[2])
#     inter_y1 = min(bbox1[3], bbox2[3])
    
#     if inter_x0 >= inter_x1 or inter_y0 >= inter_y1:
#         return 0.0
    
#     intersection_area = (inter_x1 - inter_x0) * (inter_y1 - inter_y0)
#     area1 = (bbox1[2] - bbox1[0]) * (bbox1[3] - bbox1[1])
#     area2 = (bbox2[2] - bbox2[0]) * (bbox2[3] - bbox2[1])
    
#     return intersection_area / min(area1, area2)

# def estimate_table_rows(table):
#     """Estimate number of rows in table"""
#     return table.get('rows', 3)

# def estimate_table_columns(table):
#     """Estimate number of columns in table"""
#     return table.get('cols', 2)

# def main_parallel_processing():
#     """Main function for parallel processing"""
#     pdf_path = r"D:\Upwork\POC\RAG\Data\Financial results_bajaj.pdf"
    
#     if not os.path.exists(pdf_path):
#         print(f"Error: File not found - {pdf_path}")
#         return
    
#     # Create output directory
#     timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
#     output_dir = os.path.join(r"D:\Upwork\POC\RAG\Data\output", f"parallel_processing_{timestamp}")
#     os.makedirs(output_dir, exist_ok=True)
    
#     print("🚀 Parallel Processing Options:")
#     print("1. Process all pages (parallel)")
#     print("2. Process specific page")
#     print("3. Process page range (parallel)")
    
#     choice = input("Enter your choice (1-3): ").strip()
    
#     start_time = datetime.now()
    
#     if choice == '1':
#         extracted_data = process_pdf_parallel(pdf_path, output_dir)
        
#     elif choice == '2':
#         try:
#             page_num = int(input("Enter page number: "))
#             extracted_data = process_pdf_parallel(pdf_path, output_dir, specific_page=page_num)
#         except ValueError:
#             print("Invalid page number!")
            
#     elif choice == '3':
#         try:
#             start_page = int(input("Enter start page: "))
#             end_page = int(input("Enter end page: "))
            
#             # Process page range sequentially (since parallel processes specific pages)
#             doc = fitz.open(pdf_path)
#             for page_num in range(start_page, end_page + 1):
#                 if 0 <= page_num < len(doc):
#                     process_pdf_parallel(pdf_path, output_dir, specific_page=page_num)
#             doc.close()
            
#         except ValueError:
#             print("Invalid page numbers!")
    
#     else:
#         print("Invalid choice!")
#         return
    
#     end_time = datetime.now()
#     processing_time = (end_time - start_time).total_seconds()
    
#     print(f"\n🎉 Processing completed in {processing_time:.2f} seconds!")
#     print(f"📁 Output directory: {output_dir}")
#     print("   ├── text_extraction/     (All text files)")
#     print("   └── overlay_images/      (All overlay images)")

# # Sequential processing alternative
# def process_pdf_sequential(pdf_path, output_dir, specific_page=None):
#     """Sequential processing for comparison"""
#     doc = fitz.open(pdf_path)
    
#     if specific_page is not None:
#         pages_to_process = [specific_page]
#     else:
#         pages_to_process = range(len(doc))
    
#     # Create directories
#     text_dir = os.path.join(output_dir, "text_extraction")
#     overlay_dir = os.path.join(output_dir, "overlay_images")
#     os.makedirs(text_dir, exist_ok=True)
#     os.makedirs(overlay_dir, exist_ok=True)
    
#     for page_num in pages_to_process:
#         page = doc[page_num]
        
#         # Process sequentially
#         extracted_content = extract_structured_text_from_page(page, page_num)
#         overlay_path = create_page_overlay(page, page_num, overlay_dir, extracted_content)
#         save_page_text(extracted_content, text_dir, page_num)
        
#         print(f"✅ Page {page_num} - Text: {len(extracted_content['full_text'])} chars, Overlay: {overlay_path}")
    
#     doc.close()

# if __name__ == "__main__":
#     main_parallel_processing()

In [63]:
# import fitz  # PyMuPDF
# import os
# import json
# from datetime import datetime
# import re
# from PIL import Image, ImageDraw, ImageFont
# import io
# import concurrent.futures
# from collections import defaultdict
# from langdetect import detect

# class RobustPDFProcessor:
#     def __init__(self, pdf_path, output_dir):
#         self.pdf_path = pdf_path
#         self.output_dir = output_dir
#         self.doc = fitz.open(pdf_path)
        
#         # Create organized output directories
#         self.text_dir = os.path.join(output_dir, "text_extraction")
#         self.overlay_dir = os.path.join(output_dir, "overlay_images")
        
#         for directory in [self.text_dir, self.overlay_dir]:
#             os.makedirs(directory, exist_ok=True)
        
#         # Configuration
#         self.zoom_factor = 2
#         self.min_table_area = 1000  # Increased minimum area for tables
#         self.merge_threshold = 50   # Distance threshold for merging nearby tables
        
#     def process_pdf(self, specific_page=None):
#         """Main processing function"""
#         print(f"🔍 Processing PDF: {os.path.basename(self.pdf_path)}")
#         print(f"📄 Total pages: {len(self.doc)}")
        
#         if specific_page is not None:
#             pages_to_process = [specific_page]
#         else:
#             pages_to_process = range(len(self.doc))
        
#         all_page_data = {}
        
#         # Process pages with error handling
#         for page_num in pages_to_process:
#             try:
#                 print(f"\n📖 Processing Page {page_num}...")
#                 page_data = self.process_single_page(page_num)
#                 all_page_data[page_num] = page_data
#                 print(f"✅ Page {page_num}: {len(page_data['blocks'])} blocks, {len(page_data['tables'])} tables")
#             except Exception as e:
#                 print(f"❌ Page {page_num} failed: {e}")
#                 # Create empty page data for failed pages
#                 all_page_data[page_num] = {
#                     'page_number': page_num,
#                     'blocks': [],
#                     'tables': [],
#                     'full_text': f"⚠️ Page {page_num} processing failed: {str(e)}"
#                 }
        
#         # Save combined document with proper sequencing
#         self.save_combined_document(all_page_data)
#         return all_page_data
    
#     def process_single_page(self, page_num):
#         """Process a single page with robust error handling"""
#         page = self.doc[page_num]
#         page_data = {
#             'page_number': page_num,
#             'blocks': [],
#             'tables': [],
#             'full_text': '',
#             'dimensions': (page.rect.width, page.rect.height)
#         }
        
#         try:
#             # Get page content with detailed structure
#             blocks = page.get_text("dict")
            
#             # Robust table detection with merging
#             tables = self.detect_and_merge_tables(page, blocks, page_num)
#             page_data['tables'] = tables
            
#             # Process all blocks in reading order
#             sorted_blocks = self.sort_blocks_reading_order(blocks['blocks'])
            
#             # Identify blocks that are not in tables
#             non_table_blocks = self.filter_non_table_blocks(sorted_blocks, tables)
            
#             # Process each block
#             for block in non_table_blocks:
#                 block_data = self.process_block(block, page, page_num)
#                 if block_data:
#                     page_data['blocks'].append(block_data)
            
#             # Generate sequential text
#             page_data['full_text'] = self.generate_sequential_content(page_data['blocks'], tables)
            
#             # Create overlay image
#             self.create_smart_overlay(page, page_num, page_data['blocks'], tables)
            
#             # Save individual page text
#             self.save_page_text(page_data, page_num)
            
#         except Exception as e:
#             print(f"Error processing page {page_num}: {e}")
#             # Fallback: extract simple text
#             simple_text = page.get_text("text")
#             page_data['full_text'] = f"⚠️ Advanced processing failed. Simple text extraction:\n\n{simple_text}"
        
#         return page_data
    
#     def detect_and_merge_tables(self, page, blocks, page_num):
#         """Detect tables and merge fragmented ones"""
#         # Primary detection using PyMuPDF
#         primary_tables = self.detect_tables_pymupdf(page)
        
#         # If no tables found with PyMuPDF, use alternative methods
#         if not primary_tables:
#             primary_tables = self.detect_tables_alternative(page, blocks)
        
#         # Merge nearby or overlapping tables
#         merged_tables = self.merge_nearby_tables(primary_tables, page.rect)
        
#         # Extract content for merged tables
#         final_tables = []
#         for i, table in enumerate(merged_tables):
#             try:
#                 table_content = self.extract_table_content_smart(page, table, page_num)
#                 if table_content and table_content.get('content'):
#                     table.update(table_content)
#                     final_tables.append(table)
#             except Exception as e:
#                 print(f"Error extracting table {i} on page {page_num}: {e}")
        
#         return final_tables
    
#     def detect_tables_pymupdf(self, page):
#         """PyMuPDF table detection with better error handling"""
#         tables = []
#         try:
#             if hasattr(page, 'find_tables'):
#                 table_list = page.find_tables()
#                 for i, table in enumerate(table_list):
#                     # Validate table size
#                     bbox = list(table.bbox)
#                     if self.is_valid_table_size(bbox, page.rect):
#                         tables.append({
#                             'id': f"table_{i}",
#                             'bbox': bbox,
#                             'rows': table.row_count,
#                             'cols': table.col_count,
#                             'method': 'pymupdf',
#                             'confidence': 0.9
#                         })
#         except Exception as e:
#             # Silently handle the error, we'll use alternative methods
#             pass
#         return tables
    
#     def detect_tables_alternative(self, page, blocks):
#         """Alternative table detection when PyMuPDF fails"""
#         tables = []
        
#         # Method 1: Look for large text blocks with table-like structure
#         text_blocks = [b for b in blocks['blocks'] if b.get('type') == 0]
        
#         # Group blocks that might form tables
#         potential_tables = self.find_potential_table_regions(text_blocks, page.rect)
        
#         for i, region in enumerate(potential_tables):
#             tables.append({
#                 'id': f"alt_table_{i}",
#                 'bbox': region['bbox'],
#                 'method': 'text_structure',
#                 'confidence': region.get('confidence', 0.6)
#             })
        
#         return tables
    
#     def find_potential_table_regions(self, text_blocks, page_rect):
#         """Find regions that likely contain tables"""
#         regions = []
        
#         # Group blocks by proximity
#         grouped_blocks = self.group_blocks_by_proximity(text_blocks)
        
#         for group in grouped_blocks:
#             if len(group) >= 3:  # At least 3 blocks for a table
#                 bbox = self.merge_bboxes([b['bbox'] for b in group])
                
#                 # Check if this could be a table
#                 if (self.is_valid_table_size(bbox, page_rect) and 
#                     self.has_table_characteristics(group)):
                    
#                     regions.append({
#                         'bbox': bbox,
#                         'blocks': group,
#                         'confidence': 0.7
#                     })
        
#         return regions
    
#     def group_blocks_by_proximity(self, blocks, threshold=20):
#         """Group nearby blocks that might belong to the same table"""
#         if not blocks:
#             return []
        
#         groups = []
#         used_blocks = set()
        
#         for i, block in enumerate(blocks):
#             if i in used_blocks:
#                 continue
                
#             current_group = [block]
#             used_blocks.add(i)
            
#             # Find nearby blocks
#             for j, other_block in enumerate(blocks):
#                 if j not in used_blocks and self.are_blocks_near(block, other_block, threshold):
#                     current_group.append(other_block)
#                     used_blocks.add(j)
            
#             groups.append(current_group)
        
#         return groups
    
#     def are_blocks_near(self, block1, block2, threshold):
#         """Check if two blocks are near each other"""
#         bbox1 = block1['bbox']
#         bbox2 = block2['bbox']
        
#         # Calculate horizontal and vertical distances
#         horizontal_dist = max(0, bbox1[0] - bbox2[2], bbox2[0] - bbox1[2])
#         vertical_dist = max(0, bbox1[1] - bbox2[3], bbox2[1] - bbox1[3])
        
#         return horizontal_dist < threshold and vertical_dist < threshold
    
#     def has_table_characteristics(self, blocks):
#         """Check if a group of blocks has table-like characteristics"""
#         if len(blocks) < 2:
#             return False
        
#         # Check for aligned columns
#         x_positions = []
#         for block in blocks:
#             x_positions.append(block['bbox'][0])  # Left edge
        
#         # If many blocks share similar x-positions, it might be a table
#         x_variance = len(set(round(x) for x in x_positions))
#         return x_variance < len(blocks) * 0.7  # 70% should share x-positions
    
#     def merge_nearby_tables(self, tables, page_rect):
#         """Merge tables that are close to each other"""
#         if not tables:
#             return []
        
#         merged = []
#         used_tables = set()
        
#         for i, table1 in enumerate(tables):
#             if i in used_tables:
#                 continue
                
#             current_merge = [table1]
#             used_tables.add(i)
            
#             # Find nearby tables to merge
#             for j, table2 in enumerate(tables):
#                 if j not in used_tables and self.should_merge_tables(table1, table2):
#                     current_merge.append(table2)
#                     used_tables.add(j)
            
#             # Merge all tables in current_merge
#             if len(current_merge) > 1:
#                 merged_table = self.merge_table_group(current_merge, page_rect)
#                 merged.append(merged_table)
#             else:
#                 merged.append(table1)
        
#         return merged
    
#     def should_merge_tables(self, table1, table2):
#         """Determine if two tables should be merged"""
#         bbox1 = table1['bbox']
#         bbox2 = table2['bbox']
        
#         # Calculate overlap and proximity
#         overlap = self.calculate_bbox_overlap(bbox1, bbox2)
#         distance = self.calculate_bbox_distance(bbox1, bbox2)
        
#         # Merge if they overlap significantly or are very close
#         return overlap > 0.1 or distance < self.merge_threshold
    
#     def calculate_bbox_overlap(self, bbox1, bbox2):
#         """Calculate overlap percentage between two bboxes"""
#         inter_x0 = max(bbox1[0], bbox2[0])
#         inter_y0 = max(bbox1[1], bbox2[1])
#         inter_x1 = min(bbox1[2], bbox2[2])
#         inter_y1 = min(bbox1[3], bbox2[3])
        
#         if inter_x0 >= inter_x1 or inter_y0 >= inter_y1:
#             return 0.0
        
#         intersection_area = (inter_x1 - inter_x0) * (inter_y1 - inter_y0)
#         area1 = (bbox1[2] - bbox1[0]) * (bbox1[3] - bbox1[1])
#         area2 = (bbox2[2] - bbox2[0]) * (bbox2[3] - bbox2[1])
        
#         return intersection_area / min(area1, area2)
    
#     def calculate_bbox_distance(self, bbox1, bbox2):
#         """Calculate minimum distance between two bboxes"""
#         # Horizontal distance
#         left_dist = bbox2[0] - bbox1[2]  # table2 left to table1 right
#         right_dist = bbox1[0] - bbox2[2]  # table1 left to table2 right
        
#         # Vertical distance
#         top_dist = bbox2[1] - bbox1[3]   # table2 top to table1 bottom
#         bottom_dist = bbox1[1] - bbox2[3] # table1 top to table2 bottom
        
#         horizontal_dist = max(left_dist, right_dist, 0)
#         vertical_dist = max(top_dist, bottom_dist, 0)
        
#         return max(horizontal_dist, vertical_dist)
    
#     def merge_table_group(self, tables, page_rect):
#         """Merge a group of tables into one"""
#         if not tables:
#             return None
        
#         # Combine all bboxes
#         all_bboxes = [table['bbox'] for table in tables]
#         merged_bbox = self.merge_bboxes(all_bboxes)
        
#         # Use the table with highest confidence as base
#         best_table = max(tables, key=lambda x: x.get('confidence', 0))
        
#         return {
#             'id': f"merged_{best_table['id']}",
#             'bbox': merged_bbox,
#             'rows': sum(t.get('rows', 1) for t in tables),
#             'cols': max(t.get('cols', 1) for t in tables),
#             'method': 'merged',
#             'confidence': best_table.get('confidence', 0.8),
#             'merged_count': len(tables)
#         }
    
#     def extract_table_content_smart(self, page, table, page_num):
#         """Smart table content extraction with fallbacks"""
#         try:
#             # Try structured extraction first
#             table_data = self.extract_structured_table(page, table)
#             if table_data and table_data.get('raw_data'):
#                 return table_data
            
#             # Fallback to text-based extraction
#             return self.extract_table_text_based(page, table)
            
#         except Exception as e:
#             print(f"Table extraction failed for {table['id']} on page {page_num}: {e}")
#             return {'content': '<!-- Table extraction failed -->', 'raw_data': []}
    
#     def extract_structured_table(self, page, table):
#         """Extract structured table data"""
#         try:
#             tables = page.find_tables(clip=table['bbox'])
#             if tables:
#                 table_obj = tables[0]
#                 table_data = table_obj.extract()
                
#                 if table_data and any(any(cell is not None for cell in row) for row in table_data):
#                     return {
#                         'content': self.format_table_as_markdown(table_data),
#                         'raw_data': table_data,
#                         'rows': len(table_data),
#                         'cols': len(table_data[0]) if table_data else 0
#                     }
#         except:
#             pass
        
#         return None
    
#     def extract_table_text_based(self, page, table):
#         """Extract table content from text"""
#         table_text = page.get_text("text", clip=table['bbox']).strip()
        
#         if not table_text:
#             return {'content': '<!-- Empty table -->', 'raw_data': []}
        
#         # Try to parse as table
#         lines = [line.strip() for line in table_text.split('\n') if line.strip()]
#         table_data = self.parse_text_as_simple_table(lines)
        
#         return {
#             'content': self.format_table_as_markdown(table_data) if table_data else f"```\n{table_text}\n```",
#             'raw_data': table_data if table_data else [],
#             'rows': len(table_data) if table_data else 0,
#             'cols': len(table_data[0]) if table_data else 0
#         }
    
#     def parse_text_as_simple_table(self, lines):
#         """Parse text lines as simple table data"""
#         table_data = []
        
#         for line in lines:
#             # Split by multiple spaces or consistent delimiters
#             cells = re.split(r'\s{2,}', line)
#             cells = [cell.strip() for cell in cells if cell.strip()]
            
#             if cells:
#                 table_data.append(cells)
        
#         # Only return if it looks like a table (multiple rows with similar columns)
#         if len(table_data) > 1 and max(len(row) for row in table_data) > 1:
#             return table_data
        
#         return None
    
#     def format_table_as_markdown(self, table_data):
#         """Format table data as markdown"""
#         if not table_data or not any(any(cell.strip() for cell in row) for row in table_data):
#             return "<!-- Empty table -->\n"
        
#         # Clean data
#         cleaned_data = []
#         for row in table_data:
#             cleaned_row = []
#             for cell in row:
#                 cell_text = str(cell).strip() if cell is not None else ""
#                 cell_text = re.sub(r'\s+', ' ', cell_text)
#                 cleaned_row.append(cell_text)
#             cleaned_data.append(cleaned_row)
        
#         # Remove completely empty rows
#         cleaned_data = [row for row in cleaned_data if any(cell.strip() for cell in row)]
        
#         if not cleaned_data:
#             return "<!-- Empty table after cleaning -->\n"
        
#         # Create markdown table
#         markdown_lines = []
        
#         # Header row (use first row as header)
#         header_row = cleaned_data[0]
#         markdown_lines.append("| " + " | ".join(header_row) + " |")
        
#         # Separator row
#         separator = "| " + " | ".join(["---"] * len(header_row)) + " |"
#         markdown_lines.append(separator)
        
#         # Data rows (skip first row if it's header)
#         for row in cleaned_data[1:]:
#             # Align columns with header
#             aligned_row = row[:]
#             while len(aligned_row) < len(header_row):
#                 aligned_row.append("")
            
#             markdown_lines.append("| " + " | ".join(aligned_row) + " |")
        
#         return "\n".join(markdown_lines) + "\n\n"
    
#     def process_block(self, block, page, page_num):
#         """Process individual text block"""
#         if block.get('type') != 0:  # Only process text blocks
#             return None
        
#         text_content = self.extract_text_from_block(block)
#         if not text_content.strip():
#             return None
        
#         # Detect language
#         language = self.detect_language(text_content)
        
#         return {
#             'bbox': block['bbox'],
#             'text': text_content,
#             'language': language,
#             'type': 'text',
#             'subtype': self.classify_text_block(block, text_content)
#         }
    
#     def detect_language(self, text):
#         """Detect text language"""
#         try:
#             sample_text = text[:100]
#             if len(sample_text) > 10:
#                 return detect(sample_text)
#         except Exception:
#             pass
#         return 'unknown'
    
#     def extract_text_from_block(self, block):
#         """Extract text from block"""
#         text_parts = []
        
#         if 'lines' in block:
#             for line in block['lines']:
#                 line_text = ""
#                 if 'spans' in line:
#                     for span in line['spans']:
#                         span_text = span.get('text', '').strip()
#                         if span_text:
#                             line_text += span_text + " "
                
#                 line_text = line_text.strip()
#                 if line_text:
#                     text_parts.append(line_text)
        
#         return '\n'.join(text_parts)
    
#     def classify_text_block(self, block, text_content):
#         """Classify text block type"""
#         word_count = len(text_content.split())
#         lines = block.get('lines', [])
        
#         # Check font sizes for heading detection
#         max_font_size = 0
#         for line in lines:
#             for span in line.get('spans', []):
#                 max_font_size = max(max_font_size, span.get('size', 0))
        
#         if max_font_size > 16:
#             return 'heading'
#         elif word_count > 50:
#             return 'paragraph'
#         else:
#             return 'text_fragment'
    
#     def filter_non_table_blocks(self, blocks, tables):
#         """Filter out blocks that are inside table regions"""
#         non_table_blocks = []
        
#         for block in blocks:
#             block_in_table = False
#             for table in tables:
#                 if self.is_block_in_table(block['bbox'], table['bbox']):
#                     block_in_table = True
#                     break
            
#             if not block_in_table:
#                 non_table_blocks.append(block)
        
#         return non_table_blocks
    
#     def sort_blocks_reading_order(self, blocks):
#         """Sort blocks in proper reading order"""
#         def reading_order(block):
#             bbox = block.get('bbox', [0, 0, 0, 0])
#             return (bbox[1], bbox[0])  # Y then X
        
#         return sorted(blocks, key=reading_order)
    
#     def generate_sequential_content(self, blocks, tables):
#         """Generate properly sequenced content"""
#         all_elements = []
        
#         # Add text blocks
#         for block in blocks:
#             all_elements.append({
#                 'type': 'text',
#                 'content': block['text'],
#                 'bbox': block['bbox'],
#                 'y_position': block['bbox'][1]
#             })
        
#         # Add tables
#         for table in tables:
#             all_elements.append({
#                 'type': 'table',
#                 'content': table.get('content', ''),
#                 'bbox': table['bbox'],
#                 'y_position': table['bbox'][1]
#             })
        
#         # Sort by vertical position
#         all_elements.sort(key=lambda x: x['y_position'])
        
#         # Generate sequential text
#         sequential_text = []
#         for element in all_elements:
#             if element['type'] == 'text':
#                 sequential_text.append(element['content'])
#             else:
#                 sequential_text.append("\n" + element['content'])
        
#         return '\n\n'.join(sequential_text)
    
#     def create_smart_overlay(self, page, page_num, blocks, tables):
#         """Create overlay with table merging visualization"""
#         try:
#             mat = fitz.Matrix(self.zoom_factor, self.zoom_factor)
#             pix = page.get_pixmap(matrix=mat)
            
#             img_data = pix.tobytes("png")
#             img = Image.open(io.BytesIO(img_data))
            
#             overlay = Image.new("RGBA", img.size, (0, 0, 0, 0))
#             draw = ImageDraw.Draw(overlay)
            
#             try:
#                 font = ImageFont.truetype("arial.ttf", 12)
#                 small_font = ImageFont.truetype("arial.ttf", 8)
#             except:
#                 font = ImageFont.load_default()
#                 small_font = ImageFont.load_default()
            
#             # Color scheme
#             colors = {
#                 'heading': (255, 100, 100, 96),
#                 'paragraph': (100, 100, 255, 64),
#                 'text_fragment': (100, 200, 100, 48),
#                 'table': (180, 50, 230, 96),
#                 'merged_table': (230, 50, 180, 96)  # Different color for merged tables
#             }
            
#             # Draw tables
#             for i, table in enumerate(tables):
#                 bbox = [coord * self.zoom_factor for coord in table['bbox']]
                
#                 # Use different color for merged tables
#                 if table.get('method') == 'merged':
#                     color = colors['merged_table']
#                     table_label = f"MERGED TABLE {i+1}"
#                 else:
#                     color = colors['table']
#                     table_label = f"TABLE {i+1}"
                
#                 if table.get('merged_count', 0) > 1:
#                     table_label += f" ({table['merged_count']} merged)"
                
#                 draw.rectangle(bbox, fill=color, 
#                               outline=(color[0], color[1], color[2], 255), 
#                               width=3)
                
#                 text_bbox = draw.textbbox((bbox[0] + 5, bbox[1] + 5), table_label, font=font)
#                 text_bg = [text_bbox[0]-3, text_bbox[1]-3, text_bbox[2]+3, text_bbox[3]+3]
#                 draw.rectangle(text_bg, fill=(255, 255, 255, 224))
#                 draw.text((bbox[0] + 5, bbox[1] + 5), table_label, fill=(0, 0, 0, 255), font=font)
            
#             # Draw text blocks
#             for i, block in enumerate(blocks):
#                 bbox = [coord * self.zoom_factor for coord in block['bbox']]
#                 color = colors.get(block.get('subtype', 'text_fragment'), colors['text_fragment'])
                
#                 draw.rectangle(bbox, fill=color, 
#                               outline=(color[0], color[1], color[2], 255), 
#                               width=2)
                
#                 block_label = f"B{i+1}"
#                 draw.text((bbox[0] + 5, bbox[1] + 5), block_label, 
#                          fill=(0, 0, 0, 255), font=small_font)
            
#             # Add watermark
#             watermark_text = f"Page {page_num} | {len(blocks)} blocks | {len(tables)} tables"
#             watermark_bbox = draw.textbbox((10, 10), watermark_text, font=small_font)
#             draw.rectangle([watermark_bbox[0]-5, watermark_bbox[1]-5, 
#                            watermark_bbox[2]+5, watermark_bbox[3]+5], 
#                            fill=(255, 255, 255, 192))
#             draw.text((10, 10), watermark_text, fill=(0, 0, 0, 255), font=small_font)
            
#             # Combine and save
#             result = Image.alpha_composite(img.convert("RGBA"), overlay)
#             overlay_path = os.path.join(self.overlay_dir, f"page_{page_num:03d}_overlay.png")
#             result.save(overlay_path, "PNG")
            
#         except Exception as e:
#             print(f"Overlay creation failed for page {page_num}: {e}")
    
#     def save_page_text(self, page_data, page_num):
#         """Save individual page text"""
#         filename = f"page_{page_num:03d}_extracted.txt"
#         filepath = os.path.join(self.text_dir, filename)
        
#         with open(filepath, 'w', encoding='utf-8') as f:
#             f.write(f"=== PAGE {page_num} EXTRACTED TEXT ===\n\n")
#             f.write(page_data['full_text'])
        
#         print(f"💾 Saved: {filename}")
    
#     def save_combined_document(self, all_page_data):
#         """Save combined document with all pages"""
#         filename = "combined_document.txt"
#         filepath = os.path.join(self.text_dir, filename)
        
#         with open(filepath, 'w', encoding='utf-8') as f:
#             f.write("=== COMBINED DOCUMENT EXTRACTION ===\n\n")
            
#             for page_num in sorted(all_page_data.keys()):
#                 page_data = all_page_data[page_num]
#                 f.write(f"\n{'='*60}\n")
#                 f.write(f"PAGE {page_num}\n")
#                 f.write(f"{'='*60}\n\n")
#                 f.write(page_data['full_text'])
#                 f.write(f"\n\n--- End of Page {page_num} ---\n\n")
        
#         print(f"📚 Saved combined document: {filename}")
#         print(f"✅ Processed {len(all_page_data)} pages successfully!")
    
#     # Utility functions
#     def is_valid_table_size(self, bbox, page_rect):
#         """Check if table size is valid"""
#         width = bbox[2] - bbox[0]
#         height = bbox[3] - bbox[1]
#         area = width * height
        
#         # Check minimum area
#         if area < self.min_table_area:
#             return False
        
#         # Check if not too large (avoid detecting entire page as table)
#         page_area = page_rect.width * page_rect.height
#         if area > page_area * 0.8:
#             return False
        
#         return True
    
#     def is_block_in_table(self, block_bbox, table_bbox, threshold=0.3):
#         """Check if block is inside table region"""
#         inter_x0 = max(block_bbox[0], table_bbox[0])
#         inter_y0 = max(block_bbox[1], table_bbox[1])
#         inter_x1 = min(block_bbox[2], table_bbox[2])
#         inter_y1 = min(block_bbox[3], table_bbox[3])
        
#         if inter_x0 >= inter_x1 or inter_y0 >= inter_y1:
#             return False
        
#         intersection_area = (inter_x1 - inter_x0) * (inter_y1 - inter_y0)
#         block_area = (block_bbox[2] - block_bbox[0]) * (block_bbox[3] - block_bbox[1])
        
#         return intersection_area / block_area > threshold
    
#     def merge_bboxes(self, bboxes):
#         """Merge multiple bounding boxes - FIXED to handle tuples"""
#         if not bboxes:
#             return [0, 0, 0, 0]
        
#         # Convert all bboxes to lists to ensure they have copy() method
#         bbox_lists = []
#         for bbox in bboxes:
#             if isinstance(bbox, tuple):
#                 bbox_lists.append(list(bbox))
#             else:
#                 bbox_lists.append(bbox)
        
#         x0 = min(bbox[0] for bbox in bbox_lists)
#         y0 = min(bbox[1] for bbox in bbox_lists)
#         x1 = max(bbox[2] for bbox in bbox_lists)
#         y1 = max(bbox[3] for bbox in bbox_lists)
        
#         return [x0, y0, x1, y1]

#     def close(self):
#         """Close the PDF document"""
#         self.doc.close()

# def main():
#     """Main execution function"""
#     pdf_path = r"D:\Upwork\POC\RAG\Data\Financial results_bajaj.pdf"
    
#     if not os.path.exists(pdf_path):
#         print(f"Error: File not found - {pdf_path}")
#         return
    
#     # Create output directory
#     timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
#     output_dir = os.path.join(r"D:\Upwork\POC\RAG\Data\output", f"robust_processing_{timestamp}")
    
#     # Initialize processor
#     processor = RobustPDFProcessor(pdf_path, output_dir)
    
#     try:
#         print("🚀 Robust PDF Processing Started")
#         print("=" * 50)
        
#         # Process PDF
#         results = processor.process_pdf()
        
#         print("\n🎉 Processing Completed!")
#         print(f"📁 Output Directory: {output_dir}")
#         print(f"   ├── text_extraction/     (Individual page texts + combined document)")
#         print(f"   └── overlay_images/      (Smart overlay images with merged tables)")
        
#     finally:
#         processor.close()

# if __name__ == "__main__":
#     main()

In [65]:
import fitz  # PyMuPDF
import os
import json
from datetime import datetime
import re
from PIL import Image, ImageDraw, ImageFont, ImageFilter
import io
import concurrent.futures
from collections import defaultdict
from langdetect import detect
import cv2
import numpy as np

class AdvancedPDFProcessor:
    def __init__(self, pdf_path, output_dir):
        self.pdf_path = pdf_path
        self.output_dir = output_dir
        self.doc = fitz.open(pdf_path)
        
        # Create organized output directories
        self.text_dir = os.path.join(output_dir, "text_extraction")
        self.overlay_dir = os.path.join(output_dir, "overlay_images")
        self.cleaned_dir = os.path.join(output_dir, "cleaned_pages")
        
        for directory in [self.text_dir, self.overlay_dir, self.cleaned_dir]:
            os.makedirs(directory, exist_ok=True)
        
        # Configuration
        self.zoom_factor = 3  # Higher zoom for better OCR
        self.min_table_area = 1500  # Increased minimum area for tables
        self.merge_threshold = 30   # Distance threshold for merging nearby tables
        self.noise_threshold = 0.85  # Confidence threshold for text extraction
        
    def process_pdf(self, specific_page=None):
        """Main processing function with table priority"""
        print(f"🔍 Processing PDF: {os.path.basename(self.pdf_path)}")
        print(f"📄 Total pages: {len(self.doc)}")
        
        if specific_page is not None:
            pages_to_process = [specific_page]
        else:
            pages_to_process = range(len(self.doc))
        
        all_page_data = {}
        
        # Process pages with error handling
        for page_num in pages_to_process:
            try:
                print(f"\n📖 Processing Page {page_num}...")
                page_data = self.process_single_page_priority(page_num)
                all_page_data[page_num] = page_data
                print(f"✅ Page {page_num}: {len(page_data['tables'])} tables, {len(page_data['blocks'])} text blocks")
            except Exception as e:
                print(f"❌ Page {page_num} failed: {e}")
                # Create empty page data for failed pages
                all_page_data[page_num] = {
                    'page_number': page_num,
                    'blocks': [],
                    'tables': [],
                    'full_text': f"⚠️ Page {page_num} processing failed: {str(e)}"
                }
        
        # Save combined document with proper sequencing
        self.save_combined_document(all_page_data)
        return all_page_data
    
    def process_single_page_priority(self, page_num):
        """Process page with TABLE FIRST priority"""
        page = self.doc[page_num]
        page_data = {
            'page_number': page_num,
            'blocks': [],
            'tables': [],
            'full_text': '',
            'dimensions': (page.rect.width, page.rect.height)
        }
        
        try:
            # STEP 1: Clean page image first (for scanned documents)
            cleaned_image = self.clean_page_image(page, page_num)
            
            # STEP 2: Extract tables FIRST (priority)
            tables = self.extract_tables_robust(page, cleaned_image, page_num)
            page_data['tables'] = tables
            
            # STEP 3: Extract text blocks (excluding table regions)
            blocks = page.get_text("dict")
            sorted_blocks = self.sort_blocks_reading_order(blocks['blocks'])
            non_table_blocks = self.filter_non_table_blocks(sorted_blocks, tables)
            
            # STEP 4: Process text blocks with noise filtering
            for block in non_table_blocks:
                block_data = self.process_block_clean(block, page, page_num)
                if block_data and self.is_valid_text(block_data['text']):
                    page_data['blocks'].append(block_data)
            
            # STEP 5: Generate content with table priority
            page_data['full_text'] = self.generate_priority_content(page_data['blocks'], tables)
            
            # STEP 6: Create enhanced overlay
            self.create_priority_overlay(page, page_num, page_data['blocks'], tables, cleaned_image)
            
            # Save individual page text
            self.save_clean_page_text(page_data, page_num)
            
        except Exception as e:
            print(f"Error processing page {page_num}: {e}")
            # Fallback: extract simple text with cleaning
            simple_text = self.extract_clean_text(page)
            page_data['full_text'] = f"⚠️ Advanced processing failed. Cleaned text:\n\n{simple_text}"
        
        return page_data
    
    def clean_page_image(self, page, page_num):
        """Clean scanned document noise and enhance for better OCR"""
        try:
            # Convert page to high-resolution image
            mat = fitz.Matrix(self.zoom_factor, self.zoom_factor)
            pix = page.get_pixmap(matrix=mat)
            img_data = pix.tobytes("png")
            
            # Convert to OpenCV format
            nparr = np.frombuffer(img_data, np.uint8)
            img = cv2.imdecode(nparr, cv2.IMREAD_GRAYSCALE)
            
            # Apply noise reduction
            denoised = cv2.medianBlur(img, 3)
            
            # Enhance contrast
            clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
            enhanced = clahe.apply(denoised)
            
            # Remove small noise (morphological opening)
            kernel = np.ones((2,2), np.uint8)
            cleaned = cv2.morphologyEx(enhanced, cv2.MORPH_OPEN, kernel)
            
            # Save cleaned image for debugging
            cleaned_path = os.path.join(self.cleaned_dir, f"page_{page_num:03d}_cleaned.png")
            cv2.imwrite(cleaned_path, cleaned)
            
            return cleaned
            
        except Exception as e:
            print(f"Image cleaning failed for page {page_num}: {e}")
            return None
    
    def extract_tables_robust(self, page, cleaned_image, page_num):
        """Robust table extraction with multiple fallbacks"""
        tables = []
        
        # Method 1: Try PyMuPDF table detection first
        pymupdf_tables = self.detect_tables_pymupdf(page)
        if pymupdf_tables:
            tables.extend(pymupdf_tables)
        
        # Method 2: Visual table detection from cleaned image
        if cleaned_image is not None:
            visual_tables = self.detect_tables_visual(cleaned_image, page.rect, page_num)
            tables.extend(visual_tables)
        
        # Method 3: Text structure analysis as fallback
        if not tables:
            blocks = page.get_text("dict")
            structure_tables = self.detect_tables_structure(blocks, page.rect)
            tables.extend(structure_tables)
        
        # Merge nearby tables and extract content
        merged_tables = self.merge_nearby_tables(tables, page.rect)
        final_tables = []
        
        for i, table in enumerate(merged_tables):
            try:
                table_content = self.extract_table_content_accurate(page, table, cleaned_image, page_num)
                if table_content and table_content.get('content'):
                    table.update(table_content)
                    final_tables.append(table)
            except Exception as e:
                print(f"Error extracting table {i} on page {page_num}: {e}")
        
        return final_tables
    
    def detect_tables_pymupdf(self, page):
        """PyMuPDF table detection with validation"""
        tables = []
        try:
            if hasattr(page, 'find_tables'):
                table_list = page.find_tables()
                for i, table in enumerate(table_list):
                    bbox = list(table.bbox)
                    if self.is_valid_table_size(bbox, page.rect):
                        tables.append({
                            'id': f"pymupdf_{i}",
                            'bbox': bbox,
                            'rows': table.row_count,
                            'cols': table.col_count,
                            'method': 'pymupdf',
                            'confidence': 0.9
                        })
        except Exception:
            pass  # Silently handle errors
        return tables
    
    def detect_tables_visual(self, cleaned_image, page_rect, page_num):
        """Detect tables from cleaned image using contour analysis"""
        tables = []
        try:
            # Convert pixel coordinates to PDF coordinates
            scale_x = page_rect.width / cleaned_image.shape[1]
            scale_y = page_rect.height / cleaned_image.shape[0]
            
            # Threshold image to binary
            _, binary = cv2.threshold(cleaned_image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
            
            # Find contours
            contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            
            for contour in contours:
                area = cv2.contourArea(contour)
                if area > 5000:  # Minimum area for table detection
                    x, y, w, h = cv2.boundingRect(contour)
                    
                    # Convert to PDF coordinates
                    pdf_x = x * scale_x
                    pdf_y = y * scale_y
                    pdf_w = w * scale_x
                    pdf_h = h * scale_y
                    
                    bbox = [pdf_x, pdf_y, pdf_x + pdf_w, pdf_y + pdf_h]
                    
                    if self.is_valid_table_size(bbox, page_rect):
                        tables.append({
                            'id': f"visual_{len(tables)}",
                            'bbox': bbox,
                            'method': 'visual',
                            'confidence': 0.7
                        })
                        
        except Exception as e:
            print(f"Visual table detection failed for page {page_num}: {e}")
        
        return tables
    
    def detect_tables_structure(self, blocks, page_rect):
        """Detect tables from text block structure"""
        tables = []
        text_blocks = [b for b in blocks['blocks'] if b.get('type') == 0]
        
        # Group aligned blocks
        aligned_groups = self.find_aligned_table_blocks(text_blocks)
        
        for group in aligned_groups:
            if len(group) >= 4:  # Reasonable table should have multiple cells
                bbox = self.merge_bboxes([b['bbox'] for b in group])
                if self.is_valid_table_size(bbox, page_rect):
                    tables.append({
                        'id': f"structure_{len(tables)}",
                        'bbox': bbox,
                        'method': 'structure',
                        'confidence': 0.6,
                        'block_count': len(group)
                    })
        
        return tables
    
    def find_aligned_table_blocks(self, blocks):
        """Find blocks aligned in table-like structure"""
        # Group by rows (similar y-coordinates)
        rows = defaultdict(list)
        for block in blocks:
            center_y = (block['bbox'][1] + block['bbox'][3]) / 2
            row_key = round(center_y / 10) * 10  # 10-pixel tolerance
            rows[row_key].append(block)
        
        # Find columns within rows
        table_groups = []
        for row_blocks in rows.values():
            if len(row_blocks) >= 2:
                # Sort by x-coordinate and check alignment
                row_blocks.sort(key=lambda b: b['bbox'][0])
                table_groups.append(row_blocks)
        
        return table_groups
    
    def extract_table_content_accurate(self, page, table, cleaned_image, page_num):
        """Extract table content with high accuracy"""
        try:
            # Method 1: Try structured extraction first
            structured_data = self.extract_structured_table_data(page, table)
            if structured_data and structured_data.get('raw_data'):
                return structured_data
            
            # Method 2: High-resolution text extraction
            highres_text = self.extract_highres_table_text(page, table, cleaned_image)
            if highres_text and self.is_valid_table_content(highres_text):
                parsed_data = self.parse_table_text_accurately(highres_text)
                if parsed_data:
                    return {
                        'content': self.format_clean_table(parsed_data),
                        'raw_data': parsed_data,
                        'rows': len(parsed_data),
                        'cols': len(parsed_data[0]) if parsed_data else 0
                    }
            
            return {'content': '<!-- Table content extraction failed -->', 'raw_data': []}
            
        except Exception as e:
            print(f"Accurate table extraction failed: {e}")
            return {'content': '<!-- Table extraction error -->', 'raw_data': []}
    
    def extract_structured_table_data(self, page, table):
        """Extract structured table data using PyMuPDF"""
        try:
            tables = page.find_tables(clip=table['bbox'])
            if tables:
                table_obj = tables[0]
                table_data = table_obj.extract()
                
                # Clean the extracted data
                cleaned_data = []
                for row in table_data:
                    cleaned_row = []
                    for cell in row:
                        if cell is not None:
                            cell_text = str(cell).strip()
                            # Remove noise characters
                            cell_text = re.sub(r'[~`@#$%^&*_+=|\\:;"<>?]', '', cell_text)
                            cell_text = re.sub(r'\s+', ' ', cell_text)
                            if cell_text:
                                cleaned_row.append(cell_text)
                        else:
                            cleaned_row.append("")
                    
                    if any(cleaned_row):  # Only add non-empty rows
                        cleaned_data.append(cleaned_row)
                
                if cleaned_data:
                    return {
                        'content': self.format_clean_table(cleaned_data),
                        'raw_data': cleaned_data,
                        'rows': len(cleaned_data),
                        'cols': max(len(row) for row in cleaned_data) if cleaned_data else 0
                    }
        except:
            pass
        
        return None
    
    def extract_highres_table_text(self, page, table, cleaned_image):
        """Extract high-resolution text from table region"""
        try:
            # Extract from original PDF with high zoom
            mat = fitz.Matrix(4, 4)  # Very high resolution
            clip_rect = fitz.Rect(table['bbox'])
            pix = page.get_pixmap(matrix=mat, clip=clip_rect)
            text = pix.get_text()
            
            # Clean the extracted text
            cleaned_text = self.clean_extracted_text(text)
            return cleaned_text
            
        except:
            return None
    
    def parse_table_text_accurately(self, text):
        """Parse table text with high accuracy and noise removal"""
        if not text:
            return None
        
        lines = [line.strip() for line in text.split('\n') if line.strip()]
        table_data = []
        
        for line in lines:
            # Remove noise characters before splitting
            clean_line = re.sub(r'[~`@#$%^&*_+=|\\:;"<>?]', '', line)
            clean_line = re.sub(r'\s+', ' ', clean_line).strip()
            
            if not clean_line:
                continue
            
            # Split by multiple spaces (table columns)
            cells = re.split(r'\s{2,}', clean_line)
            cells = [cell.strip() for cell in cells if cell.strip()]
            
            # Only add rows that look like table data
            if len(cells) >= 2 and any(len(cell) > 1 for cell in cells):
                table_data.append(cells)
        
        # Validate table structure
        if len(table_data) >= 2:
            return table_data
        
        return None
    
    def clean_extracted_text(self, text):
        """Clean extracted text by removing noise and garbage characters"""
        if not text:
            return ""
        
        # Remove common OCR noise patterns
        noise_patterns = [
            r'[~`@#$%^&*_+=|\\:;"<>?]',  # Special characters
            r'\b[a-zA-Z]\b',  # Single letters (often noise)
            r'\d{1}\s*\d{1}\s*\d{1}',  # Random number patterns
            r'[^\w\s\.\,\-\!\?\$\€\₹\£\@\#\%\&\*\(\)\:\;]',  # Keep only meaningful chars
        ]
        
        cleaned = text
        for pattern in noise_patterns:
            cleaned = re.sub(pattern, ' ', cleaned)
        
        # Normalize whitespace
        cleaned = re.sub(r'\s+', ' ', cleaned)
        cleaned = cleaned.strip()
        
        return cleaned
    
    def is_valid_table_content(self, text):
        """Check if extracted text looks like valid table content"""
        if not text:
            return False
        
        lines = text.split('\n')
        valid_lines = 0
        
        for line in lines:
            line = line.strip()
            if len(line) > 10 and re.search(r'[a-zA-Z]{2,}', line):  # Has meaningful text
                valid_lines += 1
        
        return valid_lines >= 2  # At least 2 valid lines
    
    def format_clean_table(self, table_data):
        """Format table data as clean markdown"""
        if not table_data or not any(any(cell.strip() for cell in row) for row in table_data):
            return "<!-- Empty table -->\n"
        
        # Clean data further
        cleaned_data = []
        for row in table_data:
            cleaned_row = []
            for cell in row:
                cell_text = str(cell).strip()
                cell_text = re.sub(r'\s+', ' ', cell_text)
                cleaned_row.append(cell_text)
            if any(cleaned_row):
                cleaned_data.append(cleaned_row)
        
        if not cleaned_data:
            return "<!-- Empty table after cleaning -->\n"
        
        # Create markdown table
        markdown_lines = []
        
        # Header row
        header_row = cleaned_data[0]
        markdown_lines.append("| " + " | ".join(header_row) + " |")
        
        # Separator
        separator = "| " + " | ".join(["---"] * len(header_row)) + " |"
        markdown_lines.append(separator)
        
        # Data rows
        for row in cleaned_data[1:]:
            aligned_row = row[:len(header_row)] if len(row) > len(header_row) else row
            while len(aligned_row) < len(header_row):
                aligned_row.append("")
            
            markdown_lines.append("| " + " | ".join(aligned_row) + " |")
        
        return "\n".join(markdown_lines) + "\n\n"
    
    def process_block_clean(self, block, page, page_num):
        """Process text block with noise filtering"""
        if block.get('type') != 0:
            return None
        
        text_content = self.extract_text_from_block(block)
        clean_text = self.clean_extracted_text(text_content)
        
        if not clean_text or not self.is_valid_text(clean_text):
            return None
        
        return {
            'bbox': block['bbox'],
            'text': clean_text,
            'type': 'text',
            'subtype': self.classify_text_block(block, clean_text)
        }
    
    def is_valid_text(self, text):
        """Check if text is valid (not noise)"""
        if not text or len(text.strip()) < 3:
            return False
        
        # Check for meaningful content (not just special characters)
        if not re.search(r'[a-zA-Z]{2,}', text) and not re.search(r'\d{3,}', text):
            return False
        
        # Check for common noise patterns
        noise_patterns = [
            r'^[~`@#$%^&*_+=|\\:;"<>?\s]+$',
            r'^\d\s*\d\s*\d$',
            r'^[a-zA-Z]\s*[a-zA-Z]\s*[a-zA-Z]$'
        ]
        
        for pattern in noise_patterns:
            if re.match(pattern, text):
                return False
        
        return True
    
    def extract_clean_text(self, page):
        """Extract and clean text from page as fallback"""
        try:
            text = page.get_text("text")
            return self.clean_extracted_text(text)
        except:
            return "Text extraction failed"
    
    def extract_text_from_block(self, block):
        """Extract text from block"""
        text_parts = []
        
        if 'lines' in block:
            for line in block['lines']:
                line_text = ""
                if 'spans' in line:
                    for span in line['spans']:
                        span_text = span.get('text', '').strip()
                        if span_text:
                            line_text += span_text + " "
                
                line_text = line_text.strip()
                if line_text:
                    text_parts.append(line_text)
        
        return '\n'.join(text_parts)
    
    def classify_text_block(self, block, text_content):
        """Classify text block type"""
        word_count = len(text_content.split())
        
        if word_count > 80:
            return 'paragraph'
        elif word_count > 20:
            return 'text_fragment'
        else:
            return 'heading'
    
    def filter_non_table_blocks(self, blocks, tables):
        """Filter out blocks that are inside table regions"""
        non_table_blocks = []
        
        for block in blocks:
            block_in_table = False
            for table in tables:
                if self.is_block_in_table(block['bbox'], table['bbox']):
                    block_in_table = True
                    break
            
            if not block_in_table:
                non_table_blocks.append(block)
        
        return non_table_blocks
    
    def sort_blocks_reading_order(self, blocks):
        """Sort blocks in reading order"""
        def reading_order(block):
            bbox = block.get('bbox', [0, 0, 0, 0])
            return (bbox[1], bbox[0])
        
        return sorted(blocks, key=reading_order)
    
    def generate_priority_content(self, blocks, tables):
        """Generate content with TABLE FIRST priority"""
        all_elements = []
        
        # Add tables FIRST (priority)
        for table in tables:
            all_elements.append({
                'type': 'table',
                'content': table.get('content', ''),
                'bbox': table['bbox'],
                'y_position': table['bbox'][1]
            })
        
        # Add text blocks AFTER tables
        for block in blocks:
            all_elements.append({
                'type': 'text',
                'content': block['text'],
                'bbox': block['bbox'],
                'y_position': block['bbox'][1]
            })
        
        # Sort by vertical position
        all_elements.sort(key=lambda x: x['y_position'])
        
        # Generate sequential text
        sequential_text = []
        for element in all_elements:
            if element['type'] == 'table':
                sequential_text.append("📊 TABLE:\n" + element['content'])
            else:
                sequential_text.append(element['content'])
        
        return '\n\n'.join(sequential_text)
    
    def create_priority_overlay(self, page, page_num, blocks, tables, cleaned_image):
        """Create overlay showing table priority"""
        try:
            mat = fitz.Matrix(self.zoom_factor, self.zoom_factor)
            pix = page.get_pixmap(matrix=mat)
            img_data = pix.tobytes("png")
            img = Image.open(io.BytesIO(img_data))
            
            overlay = Image.new("RGBA", img.size, (0, 0, 0, 0))
            draw = ImageDraw.Draw(overlay)
            
            try:
                font = ImageFont.truetype("arial.ttf", 14)
                small_font = ImageFont.truetype("arial.ttf", 10)
            except:
                font = ImageFont.load_default()
                small_font = ImageFont.load_default()
            
            # Colors - Tables get prominent colors
            colors = {
                'table': (180, 50, 230, 128),      # Bright purple for tables
                'merged_table': (230, 50, 180, 128), # Pink for merged tables
                'heading': (255, 100, 100, 64),    # Less prominent for text
                'paragraph': (100, 100, 255, 48),
                'text_fragment': (100, 200, 100, 32)
            }
            
            # Draw tables FIRST (priority visualization)
            for i, table in enumerate(tables):
                bbox = [coord * self.zoom_factor for coord in table['bbox']]
                
                if table.get('method') == 'merged':
                    color = colors['merged_table']
                    label = f"MERGE TABLE {i+1}"
                else:
                    color = colors['table']
                    label = f"TABLE {i+1}"
                
                # Thick border for tables
                draw.rectangle(bbox, fill=color, outline=(255, 255, 255, 255), width=4)
                
                # Prominent label for tables
                text_bbox = draw.textbbox((bbox[0] + 8, bbox[1] + 8), label, font=font)
                text_bg = [text_bbox[0]-4, text_bbox[1]-4, text_bbox[2]+4, text_bbox[3]+4]
                draw.rectangle(text_bg, fill=(255, 255, 255, 224))
                draw.text((bbox[0] + 8, bbox[1] + 8), label, fill=(0, 0, 0, 255), font=font)
            
            # Draw text blocks (less prominent)
            for i, block in enumerate(blocks):
                bbox = [coord * self.zoom_factor for coord in block['bbox']]
                color = colors.get(block.get('subtype', 'text_fragment'), colors['text_fragment'])
                
                draw.rectangle(bbox, fill=color, outline=(200, 200, 200, 255), width=2)
                draw.text((bbox[0] + 5, bbox[1] + 5), f"T{i+1}", 
                         fill=(0, 0, 0, 255), font=small_font)
            
            # Priority watermark
            watermark = f"Page {page_num} | TABLES: {len(tables)} (PRIORITY) | Text: {len(blocks)}"
            watermark_bbox = draw.textbbox((10, 10), watermark, font=small_font)
            draw.rectangle([watermark_bbox[0]-5, watermark_bbox[1]-5, 
                           watermark_bbox[2]+5, watermark_bbox[3]+5], 
                           fill=(255, 255, 255, 192))
            draw.text((10, 10), watermark, fill=(0, 0, 0, 255), font=small_font)
            
            # Combine and save
            result = Image.alpha_composite(img.convert("RGBA"), overlay)
            overlay_path = os.path.join(self.overlay_dir, f"page_{page_num:03d}_overlay.png")
            result.save(overlay_path, "PNG")
            
        except Exception as e:
            print(f"Overlay creation failed for page {page_num}: {e}")
    
    def save_clean_page_text(self, page_data, page_num):
        """Save cleaned page text"""
        filename = f"page_{page_num:03d}_extracted.txt"
        filepath = os.path.join(self.text_dir, filename)
        
        with open(filepath, 'w', encoding='utf-8') as f:
            f.write(f"=== PAGE {page_num} CLEANED EXTRACTION ===\n\n")
            f.write(page_data['full_text'])
        
        print(f"💾 Saved: {filename}")
    
    def save_combined_document(self, all_page_data):
        """Save combined document with all pages"""
        filename = "combined_document.txt"
        filepath = os.path.join(self.text_dir, filename)
        
        with open(filepath, 'w', encoding='utf-8') as f:
            f.write("=== COMBINED DOCUMENT EXTRACTION (TABLE PRIORITY) ===\n\n")
            
            for page_num in sorted(all_page_data.keys()):
                page_data = all_page_data[page_num]
                f.write(f"\n{'='*60}\n")
                f.write(f"PAGE {page_num}\n")
                f.write(f"{'='*60}\n\n")
                f.write(page_data['full_text'])
                f.write(f"\n\n--- End of Page {page_num} ---\n\n")
        
        print(f"📚 Saved combined document: {filename}")
        print(f"✅ Successfully processed {len(all_page_data)} pages!")
    
    # Utility functions
    def is_valid_table_size(self, bbox, page_rect):
        """Check if table size is valid"""
        width = bbox[2] - bbox[0]
        height = bbox[3] - bbox[1]
        area = width * height
        
        if area < self.min_table_area:
            return False
        
        page_area = page_rect.width * page_rect.height
        if area > page_area * 0.8:
            return False
        
        return True
    
    def is_block_in_table(self, block_bbox, table_bbox, threshold=0.3):
        """Check if block is inside table region"""
        inter_x0 = max(block_bbox[0], table_bbox[0])
        inter_y0 = max(block_bbox[1], table_bbox[1])
        inter_x1 = min(block_bbox[2], table_bbox[2])
        inter_y1 = min(block_bbox[3], table_bbox[3])
        
        if inter_x0 >= inter_x1 or inter_y0 >= inter_y1:
            return False
        
        intersection_area = (inter_x1 - inter_x0) * (inter_y1 - inter_y0)
        block_area = (block_bbox[2] - block_bbox[0]) * (block_bbox[3] - block_bbox[1])
        
        return intersection_area / block_area > threshold
    
    def merge_bboxes(self, bboxes):
        """Merge multiple bounding boxes"""
        if not bboxes:
            return [0, 0, 0, 0]
        
        bbox_lists = []
        for bbox in bboxes:
            if isinstance(bbox, tuple):
                bbox_lists.append(list(bbox))
            else:
                bbox_lists.append(bbox)
        
        x0 = min(bbox[0] for bbox in bbox_lists)
        y0 = min(bbox[1] for bbox in bbox_lists)
        x1 = max(bbox[2] for bbox in bbox_lists)
        y1 = max(bbox[3] for bbox in bbox_lists)
        
        return [x0, y0, x1, y1]
    
    def merge_nearby_tables(self, tables, page_rect):
        """Merge nearby tables"""
        if not tables:
            return []
        
        merged = []
        used = set()
        
        for i, table1 in enumerate(tables):
            if i in used:
                continue
                
            current_merge = [table1]
            used.add(i)
            
            for j, table2 in enumerate(tables):
                if j not in used and self.should_merge_tables(table1, table2):
                    current_merge.append(table2)
                    used.add(j)
            
            if len(current_merge) > 1:
                merged_table = self.merge_table_group(current_merge, page_rect)
                merged.append(merged_table)
            else:
                merged.append(table1)
        
        return merged
    
    def should_merge_tables(self, table1, table2):
        """Determine if tables should be merged"""
        bbox1 = table1['bbox']
        bbox2 = table2['bbox']
        
        overlap = self.calculate_bbox_overlap(bbox1, bbox2)
        distance = self.calculate_bbox_distance(bbox1, bbox2)
        
        return overlap > 0.1 or distance < self.merge_threshold
    
    def calculate_bbox_overlap(self, bbox1, bbox2):
        """Calculate overlap percentage"""
        inter_x0 = max(bbox1[0], bbox2[0])
        inter_y0 = max(bbox1[1], bbox2[1])
        inter_x1 = min(bbox1[2], bbox2[2])
        inter_y1 = min(bbox1[3], bbox2[3])
        
        if inter_x0 >= inter_x1 or inter_y0 >= inter_y1:
            return 0.0
        
        intersection_area = (inter_x1 - inter_x0) * (inter_y1 - inter_y0)
        area1 = (bbox1[2] - bbox1[0]) * (bbox1[3] - bbox1[1])
        
        return intersection_area / area1 if area1 > 0 else 0.0
    
    def calculate_bbox_distance(self, bbox1, bbox2):
        """Calculate distance between bboxes"""
        left_dist = bbox2[0] - bbox1[2]
        right_dist = bbox1[0] - bbox2[2]
        top_dist = bbox2[1] - bbox1[3]
        bottom_dist = bbox1[1] - bbox2[3]
        
        horizontal_dist = max(left_dist, right_dist, 0)
        vertical_dist = max(top_dist, bottom_dist, 0)
        
        return max(horizontal_dist, vertical_dist)
    
    def merge_table_group(self, tables, page_rect):
        """Merge table group"""
        if not tables:
            return None
        
        all_bboxes = [table['bbox'] for table in tables]
        merged_bbox = self.merge_bboxes(all_bboxes)
        
        best_table = max(tables, key=lambda x: x.get('confidence', 0))
        
        return {
            'id': f"merged_{best_table['id']}",
            'bbox': merged_bbox,
            'method': 'merged',
            'confidence': best_table.get('confidence', 0.8),
            'merged_count': len(tables)
        }

    def close(self):
        """Close the PDF document"""
        self.doc.close()

def main():
    """Main execution function"""
    pdf_path = r"D:\Upwork\POC\RAG\Data\Financial results_bajaj.pdf"
    
    if not os.path.exists(pdf_path):
        print(f"Error: File not found - {pdf_path}")
        return
    
    # Create output directory
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    output_dir = os.path.join(r"D:\Upwork\POC\RAG\Data\output", f"advanced_processing_{timestamp}")
    
    # Initialize processor
    processor = AdvancedPDFProcessor(pdf_path, output_dir)
    
    try:
        print("🚀 Advanced PDF Processing Started")
        print("📊 TABLE PRIORITY | 🧹 NOISE CLEANING | 🎯 HIGH ACCURACY")
        print("=" * 60)
        
        # Process PDF
        results = processor.process_pdf()
        
        print("\n🎉 Processing Completed!")
        print(f"📁 Output Directory: {output_dir}")
        print(f"   ├── text_extraction/     (Cleaned text with table priority)")
        print(f"   ├── overlay_images/      (Table-first visualization)")
        print(f"   └── cleaned_pages/       (Noise-removed page images)")
        
    finally:
        processor.close()

if __name__ == "__main__":
    main()

🚀 Advanced PDF Processing Started
📊 TABLE PRIORITY | 🧹 NOISE CLEANING | 🎯 HIGH ACCURACY
🔍 Processing PDF: Financial results_bajaj.pdf
📄 Total pages: 18

📖 Processing Page 0...
💾 Saved: page_000_extracted.txt
✅ Page 0: 1 tables, 17 text blocks

📖 Processing Page 1...
💾 Saved: page_001_extracted.txt
✅ Page 1: 0 tables, 8 text blocks

📖 Processing Page 2...
💾 Saved: page_002_extracted.txt
✅ Page 2: 0 tables, 13 text blocks

📖 Processing Page 3...
💾 Saved: page_003_extracted.txt
✅ Page 3: 1 tables, 0 text blocks

📖 Processing Page 4...
💾 Saved: page_004_extracted.txt
✅ Page 4: 1 tables, 18 text blocks

📖 Processing Page 5...
💾 Saved: page_005_extracted.txt
✅ Page 5: 1 tables, 14 text blocks

📖 Processing Page 6...
💾 Saved: page_006_extracted.txt
✅ Page 6: 1 tables, 5 text blocks

📖 Processing Page 7...
💾 Saved: page_007_extracted.txt
✅ Page 7: 2 tables, 23 text blocks

📖 Processing Page 8...
💾 Saved: page_008_extracted.txt
✅ Page 8: 0 tables, 17 text blocks

📖 Processing Page 9...
💾 Saved: